# 1. Imports and Funcions

In [11]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

# Create a Session
spark = SparkSession.builder.appName('SingleFamilyApp').master('spark://172.31.19.209:7077').getOrCreate()

## 1.1. Imports

In [1]:
import sys
from pyspark.sql.types import *
from pyspark.sql import Row, DataFrame, functions as F
from datetime import datetime, date, timedelta
from pyspark.sql.window import Window

***With the next lines of code every time we perform an action with a show function, it doesnt matter how big is the dataframe or how many columns does it have, it will print the table like a pandas table without the need of conversion and the posibility of running out of memory.***

In [2]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

## 1.2. Functions

In [3]:
def parse_closing_date(closing_date_str):
    closing_date = datetime.strptime(closing_date_str, '%m/%d/%Y').date()
    return closing_date

In [4]:
def parse_closing_date1(closing_date_str):
    closing_date = datetime.strptime(closing_date_str, '%m/%Y').date()
    return closing_date

# 2. Variables

***Dataframe Path***

In [15]:
df_acquisitions = 'hdfs://hdfs:9000/tfm/Dataframes/SingleFamily/SingleFamilyFixedRateMortgage/Acquisition/'
df_performance = 'hdfs://hdfs:9000/tfm/Dataframes/SingleFamily/SingleFamilyFixedRateMortgage/Performance/'
temp_acquisitions = 'hdfs://hdfs:9000/tfm//Dataframes/SingleFamily/temp/acquisitions/sandbox/'
temp_performance = 'hdfs://hdfs:9000/tfm/Dataframes/SingleFamily/temp/performance/sandbox/'
temp_combined = 'hdfs://hdfs:9000/tfm/Dataframes/SingleFamily/temp/combined/sandbox/'

# 3. Manage Data

## 3.1. Acquisitions File

## 3.1.1. Read Data

In [7]:
acquisitions = spark.read.parquet(df_acquisitions)

In [7]:
acquisitions.show()

+------------+--------+--------------------+-------+--------+--------+--------+--------+----+-----+------+---+--------+--------+-------+--------+--------+--------+-----+-----+------+------------+--------+-------+--------------+
|     LOAN_ID|ORIG_CHN|         Seller_Name|ORIG_RT|ORIG_AMT|ORIG_TRM|ORIG_DTE|FRST_DTE|OLTV|OCLTV|NUM_BO|DTI|CSCORE_B|FTHB_FLG|PURPOSE|PROP_TYP|NUM_UNIT|OCC_STAT|STATE|ZIP_3|MI_PCT|Product_Type|CSCORE_C|MI_TYPE|RELOCATION_FLG|
+------------+--------+--------------------+-------+--------+--------+--------+--------+----+-----+------+---+--------+--------+-------+--------+--------+--------+-----+-----+------+------------+--------+-------+--------------+
|100000827160|       R|               OTHER|   4.25|  150000|     360| 03/2016| 05/2016|  49|   49|     2| 27|     809|       N|      C|      SF|       1|       I|   CA|  953|  null|         FRM|     812|   null|             N|
|100002112463|       R|               OTHER|  4.625|  250000|     360| 04/2016| 06/2016|

## 3.1.2. Manage Data

***First, I am going to delete the unnecessary Acquisition variables from the dataframe***

In [8]:
acquisitions = acquisitions.drop('Seller_Name', 'Product_Type')

***Calculation of the minimum FICO score by comparing the borrower credit score vs the coborrower credit score***

In [9]:
acquisitions = acquisitions.where(~((F.col('CSCORE_B').isNull()) & (F.col('CSCORE_C').isNull())))

In [10]:
acquisitions = acquisitions.withColumn('CSCORE_MN', F.when(F.col('CSCORE_B').isNull(), F.col('CSCORE_C'))\
                          .otherwise(F.when(F.col('CSCORE_C').isNull(), F.col('CSCORE_B'))\
                          .otherwise(F.when(F.col('CSCORE_C') < F.col('CSCORE_B'), F.col('CSCORE_C'))\
                          .otherwise(F.when(F.col('CSCORE_B') < F.col('CSCORE_C'), F.col('CSCORE_B')).otherwise(F.col('CSCORE_B'))))))

In [11]:
acquisitions = acquisitions.withColumn('CSCORE_B', F.when(F.col('CSCORE_B').isNull(), F.col('CSCORE_C')).otherwise(F.col('CSCORE_B')))\
                           .withColumn('CSCORE_C', F.when(F.col('CSCORE_C').isNull(), F.col('CSCORE_B')).otherwise(F.col('CSCORE_C')))

***Calculation of the House Price***

In [12]:
acquisitions = acquisitions.withColumn('ORIG_VAL', F.lit(F.round(F.col('ORIG_AMT')/(F.col('OLTV')/100),1)))

***Replacement of missing OCLTV with OLTV values***

In [13]:
acquisitions = acquisitions.withColumn('OCLTV', F.when(F.col('OCLTV').isNull(), F.col('OLTV')).otherwise(F.col('OCLTV')))

## 3.2. Performance File

## 3.2.1. Read Data

In [14]:
performance = spark.read.parquet(df_performance)

In [15]:
performance.show()

+------------+---------------+-------------+-------+--------+--------+-------------------+----------------+-------------+-----+-----------+--------+-------------+------+-------+-------+-------+--------+-------+-------+-------+--------+--------+--------+---------+-------+-----------+------------------+----------+-----------------+------------+
|     LOAN_ID|Monthly_Rpt_Prd|Servicer_Name|LAST_RT|LAST_UPB|Loan_Age|Months_To_Legal_Mat|Adj_Month_To_Mat|Maturity_Date|  MSA|Delq_Status|MOD_FLAG|Zero_Bal_Code|ZB_DTE|LPI_DTE|FCC_DTE|DISP_DT|FCC_COST|PP_COST|AR_COST|IE_COST|TAX_COST|NS_PROCS|CE_PROCS|RMW_PROCS|O_PROCS|NON_INT_UPB|PRIN_FORG_UPB_FHFA|REPCH_FLAG|PRIN_FORG_UPB_OTH|TRANSFER_FLG|
+------------+---------------+-------------+-------+--------+--------+-------------------+----------------+-------------+-----+-----------+--------+-------------+------+-------+-------+-------+--------+-------+-------+-------+--------+--------+--------+---------+-------+-----------+------------------+--------

In [16]:
performance.count()

298507297

## 3.2.2. Manage Data

***First, we order the data by Loan ID and Monthly Reporting Period***

In [17]:
performance = performance.orderBy('LOAN_ID', F.substring(F.col('Monthly_Rpt_Prd'),7,4), F.substring(F.col('Monthly_Rpt_Prd'),1,2), F.substring(F.col('Monthly_Rpt_Prd'),4,2))

***Calculation of the total of principle forgiveness UPB***

In [18]:
performance = performance.withColumn('PRIN_FORG_UPB_OTH', F.col('PRIN_FORG_UPB_OTH').cast('double'))

In [19]:
performance = performance.fillna(0.0, ['PRIN_FORG_UPB_FHFA', 'PRIN_FORG_UPB_OTH'])

In [20]:
performance = performance.withColumn('PRIN_FORG_UPB', F.col('PRIN_FORG_UPB_FHFA') + F.col('PRIN_FORG_UPB_OTH')).drop('PRIN_FORG_UPB_FHFA', 'PRIN_FORG_UPB_OTH')

***Calculation of the Standarize Delinquenzy Status Codes***

In [21]:
performance = performance.withColumn('Delq_Status', F.when(F.col('Delq_Status') == 'X', '999').otherwise(F.col('Delq_Status')))

***Now we add the original rate from the Acquisitions File***

In [22]:
performance = performance.join(acquisitions.select('LOAN_ID', 'ORIG_RT'), 'LOAN_ID', 'left')

***Now we need to apply a function to fill current UPBs and NON_INT_UPBs***

In [23]:
performance = performance.withColumn("LAST_UPB", F.first('LAST_UPB', True).over(Window.partitionBy('LOAN_ID')\
.orderBy(F.substring(F.col('Monthly_Rpt_Prd'),7,4), F.substring(F.col('Monthly_Rpt_Prd'),1,2), F.substring(F.col('Monthly_Rpt_Prd'),4,2)).rowsBetween(0,sys.maxsize)))\
            .fillna(0.0, 'LAST_UPB')

In [24]:
performance = performance.withColumn("NON_INT_UPB", F.first('NON_INT_UPB', True).over(Window.partitionBy('LOAN_ID')\
.orderBy(F.substring(F.col('Monthly_Rpt_Prd'),7,4), F.substring(F.col('Monthly_Rpt_Prd'),1,2), F.substring(F.col('Monthly_Rpt_Prd'),4,2)).rowsBetween(0,sys.maxsize)))\
            .fillna(0.0, 'NON_INT_UPB')

***Persist in order to gain performance***

In [25]:
performance.write.parquet(temp_performance + '/1/', mode = 'overwrite')

In [26]:
performance = spark.read.parquet(temp_performance + '/1/')

In [27]:
performance_grouped = performance.withColumn('MAT_UNIQUE', F.dense_rank().over(Window.partitionBy('LOAN_ID').orderBy('Maturity_Date')))\
                .groupBy('LOAN_ID', 'Maturity_Date').agg(F.sum('MAT_UNIQUE').alias('MAT_UNIQUE'))\
                        .withColumn('MODTRM_CHNG', F.when(F.col('MAT_UNIQUE') == 1, 1).otherwise(0))

In [28]:
performance = performance.join(performance_grouped.select('LOAN_ID', 'Maturity_Date', 'MODTRM_CHNG'), ['LOAN_ID', 'Maturity_Date'], 'left')

In [29]:
performance = performance.withColumn('NON_INT_UPB', F.col('NON_INT_UPB')*(-1))\
            .withColumn('NON_INT_UPB', F.when(F.col('NON_INT_UPB') == -0.0, F.lit(0.0)).otherwise(F.col('NON_INT_UPB')))

In [30]:
performance = performance.withColumn('PRIN_FORG_UPB', F.col('PRIN_FORG_UPB')*(-1))

In [31]:
performance = performance.withColumn('PREVIOUS_UPB', F.lag('LAST_UPB').over(Window.partitionBy('LOAN_ID')\
                                        .orderBy(F.substring(F.col('Monthly_Rpt_Prd'),7,4), F.substring(F.col('Monthly_Rpt_Prd'),1,2), F.substring(F.col('Monthly_Rpt_Prd'),4,2))))\
                         .withColumn('MODUPB_CHNG', F.when((F.col('LAST_UPB') > F.col('PREVIOUS_UPB')) & (F.col('MOD_FLAG') == 'Y'), F.lit(1)).otherwise(0)).drop('PREVIOUS_UPB')

In [32]:
performance = performance.withColumn('FIN_UPB', F.col('LAST_UPB') + F.col('NON_INT_UPB') + F.col('PRIN_FORG_UPB'))

In [33]:
performance = performance.withColumn('modir_cost', F.when(F.col('MOD_FLAG') == 'Y', F.lit(F.round(((F.col('ORIG_RT')-F.col('LAST_RT'))/1200)*F.col('LAST_UPB'), 2))).otherwise(F.lit(0)))

In [34]:
performance = performance.withColumn('modfb_cost', F.when(F.col('MOD_FLAG') == 'Y', F.lit(F.round(((F.col('LAST_RT')*(-1))/1200)*F.col('NON_INT_UPB'), 2))).otherwise(F.lit(0)))

In [35]:
performance = performance.withColumn('modfg_cost', F.when((F.col('PRIN_FORG_UPB') > 0), F.col('PRIN_FORG_UPB')*(-1)).otherwise(F.lit(0*(-1))))

In [36]:
performance = performance.withColumn('c_modir_cost', F.round(F.sum('modir_cost').over(Window.partitionBy('LOAN_ID')\
                            .orderBy(F.substring(F.col('Monthly_Rpt_Prd'),7,4), F.substring(F.col('Monthly_Rpt_Prd'),1,2), F.substring(F.col('Monthly_Rpt_Prd'),4,2))\
                              .rangeBetween(Window.unboundedPreceding, 0)), 2))

In [37]:
performance = performance.withColumn('c_modfb_cost', F.round(F.sum('modfb_cost').over(Window.partitionBy('LOAN_ID')\
                            .orderBy(F.substring(F.col('Monthly_Rpt_Prd'),7,4), F.substring(F.col('Monthly_Rpt_Prd'),1,2), F.substring(F.col('Monthly_Rpt_Prd'),4,2))\
                              .rangeBetween(Window.unboundedPreceding, 0)), 2))

***Persist in order to gain performance***

In [38]:
performance.write.parquet(temp_performance + '/2/', mode = 'overwrite')

In [39]:
performance = spark.read.parquet(temp_performance + '/2/')

***Now we are going to count the number of months a LOAN is active***

In [40]:
performance_grouped = performance.groupBy('LOAN_ID').agg(F.count(F.substring(F.col('Monthly_Rpt_Prd'),1,2)).alias('Count'))

In [41]:
performance = performance.join(performance_grouped.select('LOAN_ID', 'Count'), ['LOAN_ID'], 'left')

***Obtain the date of the first time each loan was modified***

In [42]:
performance = performance.withColumn('FMOD_DTE', F.lit(F.first('Monthly_Rpt_Prd').over(Window.partitionBy('LOAN_ID')\
        .orderBy(F.desc('MOD_FLAG')))))\
                         .withColumn('FMOD_UPB', F.lit(F.first('LAST_UPB').over(Window.partitionBy('LOAN_ID')\
        .orderBy(F.desc('MOD_FLAG')))))

***Obtain the date and UPB of each loan's first credit event***

In [43]:
performance = performance.withColumn('x', F.when((F.col('Zero_Bal_Code') == '03') | 
                                                  (F.col('Zero_Bal_Code') == '09') |
                                                  ((F.col('Delq_Status') < 999) &
                                                   (F.col('Delq_Status') >= 6)), F.lit(1)).otherwise(0))\
                    .withColumn('FCE_DTE',F.lit(F.first('Monthly_Rpt_Prd').over(Window.partitionBy('LOAN_ID').orderBy(F.desc('x')))))\
                    .withColumn('FCE_UPB',F.lit(F.first('LAST_UPB').over(Window.partitionBy('LOAN_ID').orderBy(F.desc('x')))))\
                    .withColumn('SPDelq1',F.lit(F.first('Delq_Status').over(Window.partitionBy('LOAN_ID').orderBy(F.desc('x')))))\
                    .withColumn('CountFC',F.lit(F.first('Count').over(Window.partitionBy('LOAN_ID').orderBy(F.desc('x'))))).drop('x')
    

***Obtain the date and UPB of each loan's becoming 180 days delinquent***

In [44]:
performance = performance.withColumn('x', F.when(((F.col('Delq_Status') < 999) &
                                                   (F.col('Delq_Status') >= 6)), F.lit(1)).otherwise(0))\
                    .withColumn('F180_DTE',F.lit(F.first('Monthly_Rpt_Prd').over(Window.partitionBy('LOAN_ID').orderBy(F.desc('x')))))\
                    .withColumn('F180_UPB',F.lit(F.first('LAST_UPB').over(Window.partitionBy('LOAN_ID').orderBy(F.desc('x')))))\
                    .withColumn('SPDelq2',F.lit(F.first('Delq_Status').over(Window.partitionBy('LOAN_ID').orderBy(F.desc('x')))))\
                    .withColumn('CountF1',F.lit(F.first('Count').over(Window.partitionBy('LOAN_ID').orderBy(F.desc('x'))))).drop('x')
    

***Define the last status of a loan***

In [45]:
performance = performance.withColumn('LAST_STAT', F.when(F.col('Zero_Bal_Code') == '01', F.lit('P'))\
                              .otherwise(F.when(F.col('Zero_Bal_Code') == '02', F.lit('T'))\
                              .otherwise(F.when(F.col('Zero_Bal_Code') == '03', F.lit('S'))\
                              .otherwise(F.when(F.col('Zero_Bal_Code') == '06', F.lit('R'))\
                              .otherwise(F.when(F.col('Zero_Bal_Code') == '09', F.lit('F'))\
                              .otherwise(F.when(F.col('Zero_Bal_Code') == '15', F.lit('N'))\
                              .otherwise(F.when(F.col('Zero_Bal_Code') == '16', F.lit('L'))\
                              .otherwise(F.when(F.col('Delq_Status') == '999', F.lit('X'))\
                              .otherwise(F.when(F.col('Delq_Status') > 9, F.lit('9'))\
                              .otherwise(F.when(F.col('Delq_Status') == 0, F.lit('C')).otherwise(F.col('Delq_Status'))))))))))))

In [46]:
performance.write.parquet(temp_performance + '/3/', mode = 'overwrite')

In [47]:
performance = spark.read.parquet(temp_performance + '/3/')

***Now we summarize the performance dataset by keeping the last row of a loan's activity***

In [48]:
performance_last = performance.withColumn('last', F.lit(F.last('Monthly_Rpt_Prd').over(Window.partitionBy('LOAN_ID'))))\
                                .where(F.col('Monthly_Rpt_Prd') == F.col('last')).drop('last')

***Persist in order to gain performance***

In [49]:
performance_last.count()

9366882

In [50]:
udf_date = F.udf(lambda x: parse_closing_date(x), DateType())

In [51]:
udf_date1 = F.udf(lambda x: parse_closing_date1(x), DateType())

In [52]:
performance_last = performance_last.fillna('01/01/9999', ['LPI_DTE', 'FCC_DTE', 'DISP_DT']).fillna('01/9999', ['ZB_DTE'])\
                         .withColumn('DISP_DT', udf_date(F.col('DISP_DT')))\
                         .withColumn('LPI_DTE', udf_date(F.col('LPI_DTE')))\
                         .withColumn('FCC_DTE', udf_date(F.col('FCC_DTE')))\
                         .withColumn('ZB_DTE', udf_date1(F.col('ZB_DTE')))\
                         .withColumn('Monthly_Rpt_Prd', udf_date(F.col('Monthly_Rpt_Prd')))

***Calculate the months between Last Paid Installment and Disposition date (for Lost Interest Calculation)***

In [53]:
performance_last = performance_last.withColumn('lpi2disp', F.months_between(F.col('DISP_DT'), F.col('LPI_DTE')).cast('integer'))\
                                   .withColumn('lpi2disp', F.when(F.col('lpi2disp') < 0, F.col('lpi2disp')*(-1))\
                                                .otherwise(F.when(F.col('lpi2disp') > 100, F.lit('Nulo'))))\
                                   .withColumn('zb2disp', F.months_between(F.col('DISP_DT'), F.col('ZB_DTE')).cast('integer'))\
                                   .withColumn('zb2disp', F.when(F.col('zb2disp') < 0, F.col('zb2disp')*(-1))\
                                                .otherwise(F.when((F.col('zb2disp') > 100) |
                                                                  (F.col('zb2disp').isNull()), F.lit('Nulo')).otherwise(F.col('zb2disp'))))

In [54]:
performance_last.write.parquet(temp_performance + '/4/', mode = 'overwrite')

In [55]:
performance_last = spark.read.parquet(temp_performance + '/4/')

***Calculate Interest Cost, total expenses and total proceeds***

In [56]:
performance_last = performance_last.withColumn('INT_COST', F.lit(F.col('FIN_UPB')*(((F.col('LAST_RT')/100) - 0.0035)/12)*F.col('lpi2disp')))\
                                   .withColumn('INT_COST', F.when(F.col('INT_COST') < 0, F.lit(0)).otherwise(F.col('INT_COST')))

In [57]:
performance_last = performance_last.fillna(0.0, ['PP_COST', 
                                            'AR_COST', 
                                            'TAX_COST',
                                            'FCC_COST',
                                            'IE_COST']).withColumn('total_expense', F.col('FCC_COST') + 
                                                                                   F.col('PP_COST') + 
                                                                                   F.col('AR_COST') + 
                                                                                   F.col('TAX_COST') + 
                                                                                   F.col('IE_COST'))

In [58]:
performance_last = performance_last.fillna(0.0, ['NS_PROCS', 
                                                 'CE_PROCS', 
                                                 'RMW_PROCS',
                                                 'O_PROCS']).withColumn('total_proceeds', ((F.col('NS_PROCS') + 
                                                                                             F.col('CE_PROCS') + 
                                                                                             F.col('RMW_PROCS') + 
                                                                                             F.col('O_PROCS'))*(-1)))

***Calculate the Net Loss, Net Severity, Total Costs, Total Proceeds and Liquidation Expenses. Define Last Date variable***

In [59]:
performance_last = performance_last.fillna(0.0, ['LAST_UPB', 
                                                 'INT_COST', 
                                                 'total_expense',
                                                 'total_proceeds']).withColumn('NET_LOSS', F.lit(F.col('LAST_UPB') + 
                                                                                                 F.col('INT_COST') + 
                                                                                                 F.col('total_expense') + 
                                                                                                 F.col('total_proceeds')))

In [60]:
performance_last = performance_last.fillna(0.0, ['LAST_UPB', 
                                                 'INT_COST', 
                                                 'total_expense',
                                                 'total_proceeds']).withColumn('NET_SEV',F.lit((F.col('LAST_UPB') + 
                                                                                             F.col('INT_COST') + 
                                                                                             F.col('total_expense') + 
                                                                                             F.col('total_proceeds'))/F.col('LAST_UPB')))

In [61]:
performance_last = performance_last.fillna(0.0, ['LAST_UPB', 
                                                 'INT_COST', 
                                                 'FCC_COST',
                                                 'PP_COST',
                                                 'AR_COST',
                                                 'IE_COST',
                                                 'TAX_COST']).withColumn('Total_Cost', F.lit(F.col('LAST_UPB') + 
                                                                                             F.col('INT_COST') + 
                                                                                             F.col('FCC_COST') + 
                                                                                             F.col('PP_COST') + 
                                                                                             F.col('AR_COST') + 
                                                                                             F.col('IE_COST') + 
                                                                                             F.col('TAX_COST')))

In [62]:
performance_last = performance_last.fillna(0.0, ['NS_PROCS', 
                                                 'CE_PROCS', 
                                                 'RMW_PROCS',
                                                 'O_PROCS']).withColumn('Tot_Procs', F.lit(F.col('NS_PROCS') + 
                                                                                         F.col('CE_PROCS') + 
                                                                                         F.col('RMW_PROCS') + 
                                                                                         F.col('O_PROCS')))

In [63]:
performance_last = performance_last.fillna(0.0, ['FCC_COST', 
                                                 'PP_COST', 
                                                 'AR_COST',
                                                 'IE_COST',
                                                 'TAX_COST']).withColumn('Tot_Liq_Ex', F.lit(F.col('FCC_COST') + 
                                                                                     F.col('PP_COST') + 
                                                                                     F.col('AR_COST') + 
                                                                                     F.col('IE_COST') + 
                                                                                     F.col('TAX_COST')))

In [64]:
performance_last = performance_last.withColumn('LAST_DTE', F.col('DISP_DT'))

In [65]:
performance = performance_last.drop('Count', 
                               'Monthly_Rpt_Prd', 
                               'ZB_DTE', 
                               'ORIG_RT', 
                               'Servicer_Name', 
                               'Loan_Age', 
                               'Months_To_Legal_Mat', 
                               'Adj_Month_To_Mat', 
                               'Maturity_Date', 
                               'Delq_Status', 
                               'total_expense', 
                               'total_proceeds', 
                               'lpi2disp')

***Merge together full Acquisitions and Performance Data***

In [66]:
combined_data = acquisitions.join(performance, 'LOAN_ID', 'full')

In [67]:
combined_data.write.parquet(temp_combined, mode = 'overwrite')

In [68]:
combined_data = spark.read.parquet(temp_combined)

In [69]:
combined_data.count()

9366884

***Create Vintage Year and Activity Year Attributes, set missing F180_UPB and FCE_UPB equal to ORIG_AMT if the loan goes to delinquency during first six month of loan activity***

In [70]:
combined_data = combined_data.withColumn('VinYr', F.substring(F.col('ORIG_DTE'), 4, 4))\
                             .withColumn('ActYr', F.year(F.col('LAST_DTE'))) \
                             .withColumn('DispYr', F.when(F.col('DISP_DT').isNotNull(), F.substring(F.col('DISP_DT'), 1, 4)).otherwise(F.lit('NO_DISP_DT')))

In [71]:
combined_data = combined_data.withColumn('F180_UPB', F.when((F.col('SPDelq2') == 6) &
                                                            (F.col('CountF1') <= 6) &
                                                            ((F.col('F180_UPB') == 0) |
                                                             (F.col('F180_UPB').isNull())), F.col('ORIG_AMT'))\
                                          .otherwise(F.when((F.col('F180_UPB').isNotNull()), F.col('F180_UPB')).otherwise(F.lit(0))))

In [72]:
combined_data = combined_data.withColumn('FCE_UPB', F.when((F.col('SPDelq1') == 6) &
                                                           (F.col('CountFC') <= 6) &
                                                           ((F.col('FCE_UPB') == 0) |
                                                            (F.col('FCE_UPB').isNull())), F.col('ORIG_AMT'))\
                                         .otherwise(F.when((F.col('FCE_UPB').isNotNull()), F.col('FCE_UPB')).otherwise(F.lit(0))))

***Calculate Modification Costs when loans default***

In [73]:
combined_data = combined_data.withColumn('MODIR_COST', F.when((F.col('MOD_FLAG') == 'Y') &
                                              (F.col('DISP_DT').isNotNull()), 
                                               F.lit((F.col('zb2disp')*((F.col('ORIG_RT') - F.col('LAST_RT'))/1200)*F.col('LAST_UPB')) + F.col('c_modir_cost'))).otherwise(F.lit(0)))\
                             .withColumn('MODFB_COST', F.when((F.col('MOD_FLAG') == 'Y') &
                                              ((F.col('NON_INT_UPB').isNotNull()) |
                                               (F.col('NON_INT_UPB') == 0)), 
                                               F.lit((F.col('zb2disp')*(F.col('LAST_RT')/1200)*((-1)*(F.col('NON_INT_UPB')))) + F.col('c_modfb_cost'))).otherwise(F.lit(0)))

In [74]:
combined_data = combined_data.withColumn('MODTOT_COST', F.col('modfg_cost') +
                                                        F.col('MODIR_COST') +
                                                        F.col('MODFB_COST'))

In [75]:
combined_data = combined_data.drop('SPDelq1',
                                   'SPDelq2',
                                   'CountF1',
                                   'CountFC',
                                   'modfg_cost',
                                   'modir_cost')

***Drop Loans whose Origination Process is not defined***

In [76]:
combined_data = combined_data.where(F.col('ORIG_CHN').isNotNull())

In [77]:
combined_data.write.parquet(temp_combined + '/1/', mode = 'overwrite')

In [78]:
combined_data = spark.read.parquet(temp_combined + '/1/')

# 4. Summary Statistics Step

***Create buckets for FICO scores***

In [79]:
statistics_data = combined_data.withColumn('FicoBkt', F.when(F.col('CSCORE_MN').isNull(), F.lit('MissingFICO'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 0) & 
                                           (F.col('CSCORE_MN') <= 620), F.lit('0-620'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 620) & 
                                           (F.col('CSCORE_MN') <= 660), F.lit('620-660'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 660) & 
                                           (F.col('CSCORE_MN') <= 700), F.lit('660-700'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 700) & 
                                           (F.col('CSCORE_MN') <= 740), F.lit('700-740'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 740) & 
                                           (F.col('CSCORE_MN') <= 780), F.lit('740-780'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 780), F.lit('780>')))))))))

***Loan counts cut by origination vintage and purpose***

In [80]:
vint_purpose_count = statistics_data.fillna('9999', 'VinYr')

In [81]:
vintage_years = [row.VinYr for row in vint_purpose_count.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [82]:
for vinyr in vintage_years:
    vint_purpose_count = vint_purpose_count.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.lit(1)).otherwise(F.lit(0)))

In [83]:
expr = [F.sum(x).alias(x) for x in vintage_years]

. P -> Purchase
. C -> Cash-out Refinance
. R -> No Cash-out Refinance
. U -> Refinance - Not Specified

In [84]:
vint_purpose_count.where(F.col('PURPOSE').isNotNull()).groupBy('PURPOSE').agg(*expr).toPandas()

,PURPOSE,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,U,0,0,0,2,0,0,0,0,0
1,C,0,296,26925,304665,404207,505560,468013,387631,40204
2,R,0,400,28271,310344,533408,734109,353093,184145,23120
3,P,1,49,67696,829347,921872,1058141,1083067,996572,101380


***Loan counts cut by origination vintage and occupancy***

In [85]:
vint_occupancy_count = statistics_data.fillna('9999', 'VinYr')

In [86]:
vintage_years = [row.VinYr for row in vint_occupancy_count.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [87]:
for vinyr in vintage_years:
    vint_occupancy_count = vint_occupancy_count.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.lit(1)).otherwise(F.lit(0)))

In [88]:
expr = [F.sum(x).alias(x) for x in vintage_years]

. P -> Principal
. S -> Second
. R -> Investor
. U -> Unknown

In [89]:
vint_occupancy_count.where(F.col('OCC_STAT').isNotNull()).groupBy('OCC_STAT').agg(*expr).toPandas()

,OCC_STAT,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,S,0,25,5886,68267,81772,92727,86710,73175,7229
1,I,1,10,13414,140101,162164,162365,153260,125304,12904
2,P,0,710,103592,1235990,1615551,2042718,1664203,1369869,144571


***Loan counts cut by last status***

In [90]:
statistics_data.where(F.col('VinYr') == '2014').select('LOAN_ID').distinct().count()

1444358

In [91]:
vint_laststat_count = statistics_data

. C -> Current
. 1 -> 30 day delinquency
. 2 -> 60 day delinquency
. 3 -> 90 day delinquency
. 4 -> 120 day delinquency
. 5 -> 150 day delinquency
. L -> Reperforming Loan Sale
. N -> Note Sale
. P -> Prepaid
. R -> Repurchase
. T -> Third Party Sale
. X -> Unknown

In [92]:
vint_laststat_count.orderBy('LAST_STAT').where(F.col('LAST_STAT').isNotNull()).groupBy('LAST_STAT').count().toPandas()

,LAST_STAT,count
0,1,53806
1,2,13075
2,3,8474
3,4,8924
4,5,36085
5,C,6734489
6,L,44
7,N,137
8,P,2497933
9,R,7953


***Summary Stats for Fico, Original Amount and OLTV***

In [93]:
summary_stats = statistics_data

In [94]:
summary_stats.select(F.round(F.col('CSCORE_MN'),2).alias('Borrower Credit Score'), 
                     F.round(F.col('ORIG_AMT'),2).alias('Original Loan Amount'), 
                     F.round(F.col('OLTV'),2).alias('Original Loan To Value')).describe().toPandas()

,summary,Borrower Credit Score,Original Loan Amount,Original Loan To Value
0,count,9362518,9362518,9362518
1,mean,746.2843560888214,228490.68915007694,74.41458900265933
2,stddev,47.61092597148242,119526.62895282666,17.10783188115497
3,min,445,4000,2
4,max,850,1387000,97


***Loan counts cut by origination vintage and FICO bucket***

In [95]:
vint_FICObuckets_count = statistics_data.fillna('9999', 'VinYr')

In [96]:
vintage_years = [row.VinYr for row in vint_FICObuckets_count.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [97]:
for vinyr in vintage_years:
    vint_FICObuckets_count = vint_FICObuckets_count.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.lit(1)).otherwise(F.lit(0)))

In [98]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [99]:
vint_FICObuckets_count.where(F.col('FicoBkt').isNotNull()).orderBy('FicoBkt').groupBy('FicoBkt').agg(*expr).toPandas()

,FicoBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-620,0,6,157,1338,1587,1560,1908,1807,182
1,620-660,0,7,5963,81320,97014,105528,120538,110202,10296
2,660-700,0,41,16470,199192,238005,276186,272775,235419,24769
3,700-740,0,78,25725,301723,380587,469502,425397,359578,39952
4,740-780,0,177,36835,421840,549471,680170,546840,449498,48466
5,780>,1,436,37742,438945,592823,764864,536715,411844,41039


***Summary Statistics by Vintage***

In [100]:
summary_statistics = statistics_data.fillna('9999', 'VinYr')

In [101]:
summary_statistics = summary_statistics.where(F.col('VinYr') != '9999')\
                  .fillna(0.0, ['ORIG_AMT',
                                'CSCORE_B',
                                'CSCORE_C',
                                'OLTV',
                                'OCLTV',
                                'DTI',
                                'ORIG_RT',
                                'LAST_UPB']).groupBy('VinYr').agg(F.count('LOAN_ID').alias('Loan Count'), 
                                        F.round(F.sum('ORIG_AMT'),2).alias('Total Orig. UPB'), 
                                        F.round(F.mean('ORIG_AMT'),2).alias('Avg. Orig. UPB'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('CSCORE_B')))/F.sum('ORIG_AMT'), 2).alias('Borrower Credit Score'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('CSCORE_C')))/F.sum('ORIG_AMT'), 2).alias('Co-Borrower Credit Score'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('OLTV')))/F.sum('ORIG_AMT'), 2).alias('LTV Ratio'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('OCLTV')))/F.sum('ORIG_AMT'), 2).alias('CLTV Ratio'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('DTI')))/F.sum('ORIG_AMT'), 2).alias('DTI'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('ORIG_RT')))/F.sum('ORIG_AMT'), 2).alias('Note Rate')).orderBy('VinYr').fillna(0.0)

In [102]:
summary_statistics.toPandas()

,VinYr,Loan Count,Total Orig. UPB,Avg. Orig. UPB,Borrower Credit Score,Co-Borrower Credit Score,LTV Ratio,CLTV Ratio,DTI,Note Rate
0,2011,1,85000,85000.00,800.00,803.00,80.00,80.00,41.00,4.63
1,2012,745,119213000,160017.45,779.35,778.10,54.14,58.67,28.93,3.15
2,2013,122892,25768640000,209685.25,754.44,755.46,75.53,76.48,34.24,4.36
3,2014,1444358,310246494000,214798.89,753.46,754.30,76.37,77.22,34.18,4.30
4,2015,1859487,420932676000,226370.32,755.48,756.28,75.01,75.84,33.73,3.98
5,2016,2297810,540506873000,235226.97,757.68,758.37,73.36,74.04,33.47,3.72
6,2017,1904173,434127212000,227987.27,750.89,751.63,75.55,76.16,35.36,4.17
7,2018,1568348,367731425000,234470.55,748.78,749.57,76.69,77.20,37.49,4.75
8,2019,164704,39815572000,241740.16,748.39,749.06,77.18,77.58,37.42,4.81


***Acquisition Statistics Totals***

In [103]:
total_statistics = statistics_data.fillna('9999', 'VinYr')

In [104]:
total_statistics = total_statistics.where(F.col('VinYr') != '9999')\
                  .fillna(0.0, ['ORIG_AMT',
                                'CSCORE_B',
                                'CSCORE_C',
                                'OLTV',
                                'OCLTV',
                                'DTI',
                                'ORIG_RT',
                                'LAST_UPB']).withColumn('Total', F.lit('Total')).groupBy('Total')\
                                   .agg(F.count('LOAN_ID').alias('Loan Count'), 
                                        F.round(F.sum('ORIG_AMT'),2).alias('Total Orig. UPB'), 
                                        F.round(F.mean('ORIG_AMT'),2).alias('Avg. Orig. UPB'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('CSCORE_B')))/F.sum('ORIG_AMT'), 2).alias('Borrower Credit Score'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('CSCORE_C')))/F.sum('ORIG_AMT'), 2).alias('Co-Borrower Credit Score'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('OLTV')))/F.sum('ORIG_AMT'), 2).alias('LTV Ratio'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('OCLTV')))/F.sum('ORIG_AMT'), 2).alias('CLTV Ratio'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('DTI')))/F.sum('ORIG_AMT'), 2).alias('DTI'),
                                        F.round((F.sum(F.col('ORIG_AMT')*F.col('ORIG_RT')))/F.sum('ORIG_AMT'), 2).alias('Note Rate')).fillna(0.0)

In [105]:
total_statistics.toPandas()

,Total,Loan Count,Total Orig. UPB,Avg. Orig. UPB,Borrower Credit Score,Co-Borrower Credit Score,LTV Ratio,CLTV Ratio,DTI,Note Rate
0,Total,9362518,2139248190000,228490.69,753.52,754.28,75.23,75.92,34.78,4.15


***Merge Totals with Summary Statistics with breakout by Vintage for Full Acquisition Statistics Table***

In [106]:
total_stats = summary_statistics.union(total_statistics)

In [107]:
total_stats.toPandas()

,VinYr,Loan Count,Total Orig. UPB,Avg. Orig. UPB,Borrower Credit Score,Co-Borrower Credit Score,LTV Ratio,CLTV Ratio,DTI,Note Rate
0,2011,1,85000,85000.00,800.00,803.00,80.00,80.00,41.00,4.63
1,2012,745,119213000,160017.45,779.35,778.10,54.14,58.67,28.93,3.15
2,2013,122892,25768640000,209685.25,754.44,755.46,75.53,76.48,34.24,4.36
3,2014,1444358,310246494000,214798.89,753.46,754.30,76.37,77.22,34.18,4.30
4,2015,1859487,420932676000,226370.32,755.48,756.28,75.01,75.84,33.73,3.98
5,2016,2297810,540506873000,235226.97,757.68,758.37,73.36,74.04,33.47,3.72
6,2017,1904173,434127212000,227987.27,750.89,751.63,75.55,76.16,35.36,4.17
7,2018,1568348,367731425000,234470.55,748.78,749.57,76.69,77.20,37.49,4.75
8,2019,164704,39815572000,241740.16,748.39,749.06,77.18,77.58,37.42,4.81
9,Total,9362518,2139248190000,228490.69,753.52,754.28,75.23,75.92,34.78,4.15


***Performance Loan Counts by Vintage***

In [108]:
performance_vintcount = statistics_data.fillna('9999', 'VinYr')

In [109]:
performance_vintcount = performance_vintcount.where(F.col('VinYr') != '9999')\
                  .fillna(0.0, ['ORIG_AMT',
                                'CSCORE_B',
                                'CSCORE_C',
                                'OLTV',
                                'OCLTV',
                                'DTI',
                                'ORIG_RT',
                                'LAST_UPB']).withColumn('Active', F.when(F.col('LAST_STAT').isin('C','1','2','3','4','5','6','7','8','9'), F.lit(1)).otherwise(0))\
                                           .withColumn('Prepaid', F.when(F.col('LAST_STAT').isin('P'), F.lit(1)).otherwise(0))\
                                           .withColumn('Repurchased', F.when(F.col('LAST_STAT').isin('R'), F.lit(1)).otherwise(0))\
                                           .withColumn('Modified', F.when(F.col('MOD_FLAG').isin('Y'), F.lit(1)).otherwise(0))\
                                           .withColumn('Alternative_Disposition', F.when(F.col('LAST_STAT').isin('S', 'T', 'X'), F.lit(1)).otherwise(0))\
                                           .withColumn('Default_UPB', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Net_Loss_Rate', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(0))\
                                           .withColumn('REO_Disposition', F.when(F.col('LAST_STAT').isin('F'), F.lit(1)).otherwise(0))\
                                           .withColumn('ActiveUPB', F.when(F.col('LAST_STAT').isin('C','1','2','3','4','5','6','7','8','9'), F.col('LAST_UPB')).otherwise(F.lit(0)))\
                                          .groupBy('VinYr').agg(F.count('LOAN_ID').alias('Loan Count'), 
                                                                F.round(F.sum('ORIG_AMT'),2).alias('Total Orig. UPB'), 
                                                                F.round(F.sum('Active'),2).alias('Loan Count (Active)'), 
                                                                F.round(F.sum('ActiveUPB'),2).alias('Active UPB'), 
                                                                F.round(F.sum('Prepaid'),2).alias('Prepaid'), 
                                                                F.round(F.sum('Repurchased'),2).alias('Repurchased'), 
                                                                F.round(F.sum('Alternative_Disposition'),2).alias('Alternative Disposition'), 
                                                                F.round(F.sum('REO_Disposition'),2).alias('REO Disposition'), 
                                                                F.round(F.sum('Modified'),2).alias('Modified'), 
                                                                F.round(F.sum('Default_UPB'),2).alias('Default UPB'), 
                                                                F.round(F.sum('Net_Loss_Rate'),2).alias('Net_Loss_Rate UPB')).orderBy('VinYr').fillna(0.0)

In [110]:
performance_vintcount.toPandas()

,VinYr,Loan Count,Total Orig. UPB,Loan Count (Active),Active UPB,Prepaid,Repurchased,Alternative Disposition,REO Disposition,Modified,Default UPB,Net_Loss_Rate UPB
0,2011,1,85000,1,4.628879e+04,0,0,0,0,0,0.00,0.00
1,2012,745,119213000,432,2.738829e+07,312,1,0,0,0,0.00,0.00
2,2013,122892,25768640000,56403,8.478481e+09,66196,238,47,0,0,7324294.02,7.29
3,2014,1444358,310246494000,711789,1.160817e+11,729528,2437,539,0,0,81580852.94,79.27
4,2015,1859487,420932676000,1202714,2.286561e+11,654470,1773,477,0,0,73654174.47,79.52
5,2016,2297810,540506873000,1807960,3.807973e+11,488278,1187,357,0,0,53465354.98,72.23
6,2017,1904173,434127212000,1604047,3.375382e+11,298815,1143,147,0,0,23370902.46,52.25
7,2018,1568348,367731425000,1335928,2.901599e+11,231286,1098,29,0,0,5389707.89,20.93
8,2019,164704,39815572000,135579,3.007989e+10,29048,76,0,0,0,0.00,0.00


***Performance Loan Counts Totals***

In [111]:
performance_totalcount = statistics_data.fillna('9999', 'VinYr')

In [112]:
performance_totalcount = performance_totalcount.where(F.col('VinYr') != '9999')\
                  .fillna(0.0, ['ORIG_AMT',
                                'CSCORE_B',
                                'CSCORE_C',
                                'OLTV',
                                'OCLTV',
                                'DTI',
                                'ORIG_RT',
                                'LAST_UPB']).withColumn('Active', F.when(F.col('LAST_STAT').isin('C','1','2','3','4','5','6','7','8','9'), F.lit(1)).otherwise(0))\
                                           .withColumn('Prepaid', F.when(F.col('LAST_STAT').isin('P'), F.lit(1)).otherwise(0))\
                                           .withColumn('Repurchased', F.when(F.col('LAST_STAT').isin('R'), F.lit(1)).otherwise(0))\
                                           .withColumn('Modified', F.when(F.col('MOD_FLAG').isin('Y'), F.lit(1)).otherwise(0))\
                                           .withColumn('Alternative_Disposition', F.when(F.col('LAST_STAT').isin('S', 'T', 'X'), F.lit(1)).otherwise(0))\
                                           .withColumn('Default_UPB', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Net_Loss_Rate', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(0))\
                                           .withColumn('REO_Disposition', F.when(F.col('LAST_STAT').isin('F'), F.lit(1)).otherwise(0))\
                                           .withColumn('ActiveUPB', F.when(F.col('LAST_STAT').isin('C','1','2','3','4','5','6','7','8','9'), F.col('LAST_UPB')).otherwise(F.lit(0)))\
                                           .withColumn('Total', F.lit('Total'))\
                                          .groupBy('Total').agg(F.count('LOAN_ID').alias('Loan Count'), 
                                                                F.round(F.sum('ORIG_AMT'),2).alias('Total Orig. UPB'), 
                                                                F.round(F.sum('Active'),2).alias('Loan Count (Active)'), 
                                                                F.round(F.sum('ActiveUPB'),2).alias('Active UPB'), 
                                                                F.round(F.sum('Prepaid'),2).alias('Prepaid'), 
                                                                F.round(F.sum('Repurchased'),2).alias('Repurchased'), 
                                                                F.round(F.sum('Alternative_Disposition'),2).alias('Alternative Disposition'), 
                                                                F.round(F.sum('REO_Disposition'),2).alias('REO Disposition'), 
                                                                F.round(F.sum('Modified'),2).alias('Modified'), 
                                                                F.round(F.sum('Default_UPB'),2).alias('Default UPB'), 
                                                                F.round(F.sum('Net_Loss_Rate'),2).alias('Net_Loss_Rate UPB')).fillna(0.0)

In [113]:
performance_totalcount.toPandas()

,Total,Loan Count,Total Orig. UPB,Loan Count (Active),Active UPB,Prepaid,Repurchased,Alternative Disposition,REO Disposition,Modified,Default UPB,Net_Loss_Rate UPB
0,Total,9362518,2139248190000,6854853,1.391819e+12,2497933,7953,1596,0,0,2.447853e+08,311.5


***Performance statistics Totals***

In [114]:
total_perf = performance_vintcount.union(performance_totalcount)

In [115]:
total_perf.toPandas()

,VinYr,Loan Count,Total Orig. UPB,Loan Count (Active),Active UPB,Prepaid,Repurchased,Alternative Disposition,REO Disposition,Modified,Default UPB,Net_Loss_Rate UPB
0,2011,1,85000,1,4.628879e+04,0,0,0,0,0,0.000000e+00,0.00
1,2012,745,119213000,432,2.738829e+07,312,1,0,0,0,0.000000e+00,0.00
2,2013,122892,25768640000,56403,8.478481e+09,66196,238,47,0,0,7.324294e+06,7.29
3,2014,1444358,310246494000,711789,1.160817e+11,729528,2437,539,0,0,8.158085e+07,79.27
4,2015,1859487,420932676000,1202714,2.286561e+11,654470,1773,477,0,0,7.365417e+07,79.52
5,2016,2297810,540506873000,1807960,3.807973e+11,488278,1187,357,0,0,5.346535e+07,72.23
6,2017,1904173,434127212000,1604047,3.375382e+11,298815,1143,147,0,0,2.337090e+07,52.25
7,2018,1568348,367731425000,1335928,2.901599e+11,231286,1098,29,0,0,5.389708e+06,20.93
8,2019,164704,39815572000,135579,3.007989e+10,29048,76,0,0,0,0.000000e+00,0.00
9,Total,9362518,2139248190000,6854853,1.391819e+12,2497933,7953,1596,0,0,2.447853e+08,311.50


***Performance UPB broken out by Vintage***

In [116]:
performance_vintupb = statistics_data.fillna('9999', 'VinYr')

In [117]:
performance_vintupb = performance_vintupb.where(F.col('VinYr') != '9999')\
                  .fillna(0.0, ['ORIG_AMT',
                                'CSCORE_B',
                                'CSCORE_C',
                                'OLTV',
                                'OCLTV',
                                'DTI',
                                'ORIG_RT',
                                'LAST_UPB']).withColumn('Prepaid', F.when(F.col('LAST_STAT').isin('P'), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Repurchased', F.when(F.col('LAST_STAT').isin('R'), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Modified', F.when(F.col('MOD_FLAG').isin('Y'), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Alternative_Disposition', F.when(F.col('LAST_STAT').isin('S', 'T', 'X'), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Default_UPB', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Net_Loss_Rate', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(0))\
                                           .withColumn('REO_Disposition', F.when(F.col('LAST_STAT').isin('F'), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('ActiveUPB', F.when(F.col('LAST_STAT').isin('C','1','2','3','4','5','6','7','8','9'), F.col('LAST_UPB')).otherwise(F.lit(0)))\
                                          .groupBy('VinYr').agg(F.count('LOAN_ID').alias('Loan Count'), 
                                                                F.round(F.sum('ORIG_AMT'),2).alias('Total Orig. UPB'), 
                                                                F.round(F.sum('ActiveUPB'),2).alias('Active UPB'), 
                                                                F.round(F.sum('Prepaid'),2).alias('Prepaid UPB'), 
                                                                F.round(F.sum('Repurchased'),2).alias('Repurchased'), 
                                                                F.round(F.sum('Alternative_Disposition'),2).alias('Alternative Disposition'), 
                                                                F.round(F.sum('REO_Disposition'),2).alias('REO Disposition UPB'), 
                                                                F.round(F.sum('Modified'),2).alias('Modified'), 
                                                                F.round(F.sum('Default_UPB'),2).alias('Default UPB'), 
                                                                F.round(F.sum('Net_Loss_Rate'),2).alias('Net_Loss_Rate UPB')).orderBy('VinYr').fillna(0.0)

In [118]:
performance_vintupb.toPandas()

,VinYr,Loan Count,Total Orig. UPB,Active UPB,Prepaid UPB,Repurchased,Alternative Disposition,REO Disposition UPB,Modified,Default UPB,Net_Loss_Rate UPB
0,2011,1,85000,4.628879e+04,0.000000e+00,0.000000e+00,0.00,0.0,0.0,0.00,0.00
1,2012,745,119213000,2.738829e+07,2.471847e+07,6.476192e+04,0.00,0.0,0.0,0.00,0.00
2,2013,122892,25768640000,8.478481e+09,1.376181e+10,4.881672e+07,6257935.40,0.0,0.0,7324294.02,7.29
3,2014,1444358,310246494000,1.160817e+11,1.565497e+11,4.952677e+08,74714365.64,0.0,0.0,81580852.94,79.27
4,2015,1859487,420932676000,2.286561e+11,1.442131e+11,3.958166e+08,70349563.45,0.0,0.0,73654174.47,79.52
5,2016,2297810,540506873000,3.807973e+11,1.072077e+11,2.674192e+08,50631609.99,0.0,0.0,53465354.98,72.23
6,2017,1904173,434127212000,3.375382e+11,6.834340e+10,2.617181e+08,19883855.84,0.0,0.0,23370902.46,52.25
7,2018,1568348,367731425000,2.901599e+11,6.428793e+10,2.354778e+08,4053512.15,0.0,0.0,5389707.89,20.93
8,2019,164704,39815572000,3.007989e+10,8.825213e+09,1.831349e+07,0.00,0.0,0.0,0.00,0.00


***Performance UPB totals***

In [119]:
performance_totalupb = statistics_data.fillna('9999', 'VinYr')

In [120]:
performance_totalupb = performance_totalupb.where(F.col('VinYr') != '9999')\
                  .fillna(0.0, ['ORIG_AMT',
                                'CSCORE_B',
                                'CSCORE_C',
                                'OLTV',
                                'OCLTV',
                                'DTI',
                                'ORIG_RT',
                                'LAST_UPB']).withColumn('Prepaid', F.when(F.col('LAST_STAT').isin('P'), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Repurchased', F.when(F.col('LAST_STAT').isin('R'), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Modified', F.when(F.col('MOD_FLAG').isin('Y'), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Alternative_Disposition', F.when(F.col('LAST_STAT').isin('S', 'T', 'X'), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Default_UPB', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Net_Loss_Rate', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(0))\
                                           .withColumn('REO_Disposition', F.when(F.col('LAST_STAT').isin('F'), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('ActiveUPB', F.when(F.col('LAST_STAT').isin('C','1','2','3','4','5','6','7','8','9'), F.col('LAST_UPB')).otherwise(F.lit(0)))\
                                           .withColumn('Total', F.lit('Total'))\
                                           .groupBy('Total').agg(F.count('LOAN_ID').alias('Loan Count'), 
                                                                F.round(F.sum('ORIG_AMT'),2).alias('Total Orig. UPB'), 
                                                                F.round(F.sum('ActiveUPB'),2).alias('Active UPB'), 
                                                                F.round(F.sum('Prepaid'),2).alias('Prepaid UPB'), 
                                                                F.round(F.sum('Repurchased'),2).alias('Repurchased'), 
                                                                F.round(F.sum('Alternative_Disposition'),2).alias('Alternative Disposition'), 
                                                                F.round(F.sum('REO_Disposition'),2).alias('REO Disposition UPB'), 
                                                                F.round(F.sum('Modified'),2).alias('Modified'), 
                                                                F.round(F.sum('Default_UPB'),2).alias('Default UPB'), 
                                                                F.round(F.sum('Net_Loss_Rate'),2).alias('Net_Loss_Rate UPB')).fillna(0.0)

In [121]:
performance_totalupb.toPandas()

,Total,Loan Count,Total Orig. UPB,Active UPB,Prepaid UPB,Repurchased,Alternative Disposition,REO Disposition UPB,Modified,Default UPB,Net_Loss_Rate UPB
0,Total,9362518,2139248190000,1.391819e+12,5.632135e+11,1.722894e+09,2.258908e+08,0.0,0.0,2.447853e+08,311.5


***Performance UPB statistics Totals***

In [122]:
total_perf_upb = performance_vintupb.union(performance_totalupb)

In [123]:
total_perf_upb.toPandas()

,VinYr,Loan Count,Total Orig. UPB,Active UPB,Prepaid UPB,Repurchased,Alternative Disposition,REO Disposition UPB,Modified,Default UPB,Net_Loss_Rate UPB
0,2011,1,85000,4.628879e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.00
1,2012,745,119213000,2.738829e+07,2.471847e+07,6.476192e+04,0.000000e+00,0.0,0.0,0.000000e+00,0.00
2,2013,122892,25768640000,8.478481e+09,1.376181e+10,4.881672e+07,6.257935e+06,0.0,0.0,7.324294e+06,7.29
3,2014,1444358,310246494000,1.160817e+11,1.565497e+11,4.952677e+08,7.471437e+07,0.0,0.0,8.158085e+07,79.27
4,2015,1859487,420932676000,2.286561e+11,1.442131e+11,3.958166e+08,7.034956e+07,0.0,0.0,7.365417e+07,79.52
5,2016,2297810,540506873000,3.807973e+11,1.072077e+11,2.674192e+08,5.063161e+07,0.0,0.0,5.346535e+07,72.23
6,2017,1904173,434127212000,3.375382e+11,6.834340e+10,2.617181e+08,1.988386e+07,0.0,0.0,2.337090e+07,52.25
7,2018,1568348,367731425000,2.901599e+11,6.428793e+10,2.354778e+08,4.053512e+06,0.0,0.0,5.389708e+06,20.93
8,2019,164704,39815572000,3.007989e+10,8.825213e+09,1.831349e+07,0.000000e+00,0.0,0.0,0.000000e+00,0.00
9,Total,9362518,2139248190000,1.391819e+12,5.632135e+11,1.722894e+09,2.258908e+08,0.0,0.0,2.447853e+08,311.50


***Historical Net Loss Statistics by Vintage***

In [124]:
historical_vintnetloss = statistics_data.fillna('9999', 'VinYr')

In [125]:
historical_vintnetloss = historical_vintnetloss.where(F.col('VinYr') != '9999')\
                  .fillna(0.0, ['ORIG_AMT',
                                'ORIG_RT',
                                'LAST_UPB',
                                'INT_COS',
                                'Tot_Liq_Ex',
                                'FCC_COST',
                                'PP_COST',
                                'AR_COST',
                                'IE_COST',
                                'TAX_COST',
                                'Total_Cost',
                                'NS_PROCS',
                                'CE_PROCS',
                                'RMW_PROCS',
                                'O_PROCS',
                                'Tot_Procs',
                                'NET_LOSS']).withColumn('Default_UPB', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Interest', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('INT_COST')).otherwise(0))\
                                           .withColumn('UPB_Liquidations', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Tot_Liq', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Tot_Liq_Ex')).otherwise(0))\
                                           .withColumn('FCC', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('FCC_COST')).otherwise(0))\
                                           .withColumn('PP', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('PP_COST')).otherwise(0))\
                                           .withColumn('AR', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('AR_COST')).otherwise(0))\
                                           .withColumn('IE', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('IE_COST')).otherwise(0))\
                                           .withColumn('TAX', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('TAX_COST')).otherwise(0))\
                                           .withColumn('TotalCost', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Total_Cost')).otherwise(0))\
                                           .withColumn('NS', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NS_PROCS')).otherwise(0))\
                                           .withColumn('CE', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('CE_PROCS')).otherwise(0))\
                                           .withColumn('RMW', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('RMW_PROCS')).otherwise(0))\
                                           .withColumn('O', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('O_PROCS')).otherwise(0))\
                                           .withColumn('TotalProcs', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Tot_Procs')).otherwise(0))\
                                           .withColumn('NetL', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NET_LOSS')).otherwise(0))\
                                           .withColumn('Count', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')), F.lit(1)).otherwise(0))\
                                          .groupBy('VinYr').agg(F.sum('Count').alias('Loan Count'), 
                                                                F.round(F.sum('UPB_Liquidations'),2).alias('UPB for Liquidations'), 
                                                                F.round((F.sum('Default_UPB')*100/F.sum('ORIG_AMT')),2).alias('Default UPB % of Orig. UPB'), 
                                                                F.round((F.sum('Interest')*100/F.sum('Default_UPB')),2).alias('Interest on Delinquent Loans'), 
                                                                F.round((F.sum('Tot_Liq')*100/F.sum('Default_UPB')),2).alias('Total Liquidition Exp.'), 
                                                                F.round((F.sum('FCC')*100/F.sum('Default_UPB')),2).alias('Foreclosure Costs'), 
                                                                F.round((F.sum('PP')*100/F.sum('Default_UPB')),2).alias('Prop.Pres. Costs'), 
                                                                F.round((F.sum('AR')*100/F.sum('Default_UPB')),2).alias('Asset Recovery Costs'), 
                                                                F.round((F.sum('IE')*100/F.sum('Default_UPB')),2).alias('Miscellaneaous Holding Expenses and Credits'), 
                                                                F.round((F.sum('TAX')*100/F.sum('Default_UPB')),2).alias('Associated Taxes'), 
                                                                F.round((F.sum('TotalCost')*100/F.sum('Default_UPB')),2).alias('Total Costs'), 
                                                                F.round((F.sum('NS')*100/F.sum('Default_UPB')),2).alias('Sales Proceeds'), 
                                                                F.round((F.sum('CE')*100/F.sum('Default_UPB')),2).alias('Credit Enhancement Proceeds'), 
                                                                F.round((F.sum('RMW')*100/F.sum('Default_UPB')),2).alias('Repurchase/Make Whole Proceeds'), 
                                                                F.round((F.sum('O')*100/F.sum('Default_UPB')),2).alias('Other Proceeds'), 
                                                                F.round((F.sum('TotalProcs')*100/F.sum('Default_UPB')),2).alias('Total Proceeds'), 
                                                                F.round((F.sum('NetL')*100/F.sum('Default_UPB')),2).alias('Severity'), 
                                                                F.round((F.sum('NetL')),2).alias('Total Net Loss')).orderBy('VinYr').fillna(0.0)

In [126]:
historical_vintnetloss.toPandas()

,VinYr,Loan Count,UPB for Liquidations,Default UPB % of Orig. UPB,Interest on Delinquent Loans,Total Liquidition Exp.,Foreclosure Costs,Prop.Pres. Costs,Asset Recovery Costs,Miscellaneaous Holding Expenses and Credits,Associated Taxes,Total Costs,Sales Proceeds,Credit Enhancement Proceeds,Repurchase/Make Whole Proceeds,Other Proceeds,Total Proceeds,Severity,Total Net Loss
0,2011,0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2012,0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2013,52,7324294.02,0.03,0.0,7.68,4.04,0.44,0.00,0.73,2.47,107.68,79.43,7.75,0.00,1.92,89.10,18.58,1361014.90
3,2014,578,81580852.94,0.03,0.0,6.76,3.87,0.35,0.00,0.55,2.00,106.76,88.35,7.03,0.00,0.65,96.03,10.74,8758928.80
4,2015,503,73654174.47,0.02,0.0,6.17,3.74,0.32,0.01,0.44,1.66,106.17,86.25,5.68,0.28,0.31,92.52,13.65,10055647.74
5,2016,373,53465354.98,0.01,0.0,5.04,3.30,0.25,0.00,0.43,1.06,105.04,81.09,5.63,0.00,0.33,87.05,17.99,9617968.43
6,2017,161,23370902.46,0.01,0.0,3.61,2.53,0.25,0.00,0.26,0.57,103.61,61.77,5.02,0.00,0.05,66.84,36.77,8593086.27
7,2018,35,5389707.89,0.00,0.0,1.67,1.46,0.06,0.00,0.00,0.15,101.67,44.82,3.21,0.00,0.05,48.07,53.60,2888695.48
8,2019,0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


***Historical Net Loss Totals***

In [127]:
historical_totalnetloss = statistics_data.fillna('9999', 'VinYr')

In [128]:
historical_totalnetloss = historical_totalnetloss.where(F.col('VinYr') != '9999')\
                  .fillna(0.0, ['ORIG_AMT',
                                'ORIG_RT',
                                'LAST_UPB',
                                'INT_COS',
                                'Tot_Liq_Ex',
                                'FCC_COST',
                                'PP_COST',
                                'AR_COST',
                                'IE_COST',
                                'TAX_COST',
                                'Total_Cost',
                                'NS_PROCS',
                                'CE_PROCS',
                                'RMW_PROCS',
                                'O_PROCS',
                                'Tot_Procs',
                                'NET_LOSS']).withColumn('Default_UPB', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Interest', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('INT_COST')).otherwise(0))\
                                           .withColumn('UPB_Liquidations', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Tot_Liq', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Tot_Liq_Ex')).otherwise(0))\
                                           .withColumn('FCC', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('FCC_COST')).otherwise(0))\
                                           .withColumn('PP', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('PP_COST')).otherwise(0))\
                                           .withColumn('AR', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('AR_COST')).otherwise(0))\
                                           .withColumn('IE', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('IE_COST')).otherwise(0))\
                                           .withColumn('TAX', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('TAX_COST')).otherwise(0))\
                                           .withColumn('TotalCost', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Total_Cost')).otherwise(0))\
                                           .withColumn('NS', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NS_PROCS')).otherwise(0))\
                                           .withColumn('CE', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('CE_PROCS')).otherwise(0))\
                                           .withColumn('RMW', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('RMW_PROCS')).otherwise(0))\
                                           .withColumn('O', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('O_PROCS')).otherwise(0))\
                                           .withColumn('TotalProcs', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Tot_Procs')).otherwise(0))\
                                           .withColumn('NetL', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NET_LOSS')).otherwise(0))\
                                           .withColumn('Count', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')), F.lit(1)).otherwise(0))\
                                           .withColumn('Total', F.lit('Total'))\
                                          .groupBy('Total').agg(F.sum('Count').alias('Loan Count'), 
                                                                F.round(F.sum('UPB_Liquidations'),2).alias('UPB for Liquidations'), 
                                                                F.round((F.sum('Default_UPB')*100/F.sum('ORIG_AMT')),2).alias('Default UPB % of Orig. UPB'), 
                                                                F.round((F.sum('Interest')*100/F.sum('Default_UPB')),2).alias('Interest on Delinquent Loans'), 
                                                                F.round((F.sum('Tot_Liq')*100/F.sum('Default_UPB')),2).alias('Total Liquidition Exp.'), 
                                                                F.round((F.sum('FCC')*100/F.sum('Default_UPB')),2).alias('Foreclosure Costs'), 
                                                                F.round((F.sum('PP')*100/F.sum('Default_UPB')),2).alias('Prop.Pres. Costs'), 
                                                                F.round((F.sum('AR')*100/F.sum('Default_UPB')),2).alias('Asset Recovery Costs'), 
                                                                F.round((F.sum('IE')*100/F.sum('Default_UPB')),2).alias('Miscellaneaous Holding Expenses and Credits'), 
                                                                F.round((F.sum('TAX')*100/F.sum('Default_UPB')),2).alias('Associated Taxes'), 
                                                                F.round((F.sum('TotalCost')*100/F.sum('Default_UPB')),2).alias('Total Costs'), 
                                                                F.round((F.sum('NS')*100/F.sum('Default_UPB')),2).alias('Sales Proceeds'), 
                                                                F.round((F.sum('CE')*100/F.sum('Default_UPB')),2).alias('Credit Enhancement Proceeds'), 
                                                                F.round((F.sum('RMW')*100/F.sum('Default_UPB')),2).alias('Repurchase/Make Whole Proceeds'), 
                                                                F.round((F.sum('O')*100/F.sum('Default_UPB')),2).alias('Other Proceeds'), 
                                                                F.round((F.sum('TotalProcs')*100/F.sum('Default_UPB')),2).alias('Total Proceeds'), 
                                                                F.round((F.sum('NetL')*100/F.sum('Default_UPB')),2).alias('Severity'), 
                                                                F.round((F.sum('NetL')),2).alias('Total Net Loss')).fillna(0.0)

In [129]:
historical_totalnetloss.toPandas()

,Total,Loan Count,UPB for Liquidations,Default UPB % of Orig. UPB,Interest on Delinquent Loans,Total Liquidition Exp.,Foreclosure Costs,Prop.Pres. Costs,Asset Recovery Costs,Miscellaneaous Holding Expenses and Credits,Associated Taxes,Total Costs,Sales Proceeds,Credit Enhancement Proceeds,Repurchase/Make Whole Proceeds,Other Proceeds,Total Proceeds,Severity,Total Net Loss
0,Total,1702,2.447853e+08,0.01,0.0,5.82,3.53,0.31,0.0,0.45,1.53,105.82,82.37,6.06,0.08,0.44,88.96,16.86,41275341.62


***Historical Net Loss Union***

In [130]:
historical_total = historical_vintnetloss.union(historical_totalnetloss)

In [131]:
historical_total.toPandas()

,VinYr,Loan Count,UPB for Liquidations,Default UPB % of Orig. UPB,Interest on Delinquent Loans,Total Liquidition Exp.,Foreclosure Costs,Prop.Pres. Costs,Asset Recovery Costs,Miscellaneaous Holding Expenses and Credits,Associated Taxes,Total Costs,Sales Proceeds,Credit Enhancement Proceeds,Repurchase/Make Whole Proceeds,Other Proceeds,Total Proceeds,Severity,Total Net Loss
0,2011,0,0.000000e+00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2012,0,0.000000e+00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2013,52,7.324294e+06,0.03,0.0,7.68,4.04,0.44,0.00,0.73,2.47,107.68,79.43,7.75,0.00,1.92,89.10,18.58,1361014.90
3,2014,578,8.158085e+07,0.03,0.0,6.76,3.87,0.35,0.00,0.55,2.00,106.76,88.35,7.03,0.00,0.65,96.03,10.74,8758928.80
4,2015,503,7.365417e+07,0.02,0.0,6.17,3.74,0.32,0.01,0.44,1.66,106.17,86.25,5.68,0.28,0.31,92.52,13.65,10055647.74
5,2016,373,5.346535e+07,0.01,0.0,5.04,3.30,0.25,0.00,0.43,1.06,105.04,81.09,5.63,0.00,0.33,87.05,17.99,9617968.43
6,2017,161,2.337090e+07,0.01,0.0,3.61,2.53,0.25,0.00,0.26,0.57,103.61,61.77,5.02,0.00,0.05,66.84,36.77,8593086.27
7,2018,35,5.389708e+06,0.00,0.0,1.67,1.46,0.06,0.00,0.00,0.15,101.67,44.82,3.21,0.00,0.05,48.07,53.60,2888695.48
8,2019,0,0.000000e+00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Total,1702,2.447853e+08,0.01,0.0,5.82,3.53,0.31,0.00,0.45,1.53,105.82,82.37,6.06,0.08,0.44,88.96,16.86,41275341.62


***Historical Net Loss Statistics by Disposition Year***

In [132]:
historical_dispnetloss = statistics_data.where(F.col('DispYr') != '9999').where(F.col('DispYr') != 'NO_DISP_DT')

In [133]:
historical_dispnetloss = historical_dispnetloss\
                  .fillna(0.0, ['ORIG_AMT',
                                'ORIG_RT',
                                'LAST_UPB',
                                'INT_COS',
                                'Tot_Liq_Ex',
                                'FCC_COST',
                                'PP_COST',
                                'AR_COST',
                                'IE_COST',
                                'TAX_COST',
                                'Total_Cost',
                                'NS_PROCS',
                                'CE_PROCS',
                                'RMW_PROCS',
                                'O_PROCS',
                                'Tot_Procs',
                                'NET_LOSS']).withColumn('Default_UPB', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Interest', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('INT_COST')).otherwise(0))\
                                           .withColumn('UPB_Liquidations', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Tot_Liq', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Tot_Liq_Ex')).otherwise(0))\
                                           .withColumn('FCC', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('FCC_COST')).otherwise(0))\
                                           .withColumn('PP', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('PP_COST')).otherwise(0))\
                                           .withColumn('AR', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('AR_COST')).otherwise(0))\
                                           .withColumn('IE', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('IE_COST')).otherwise(0))\
                                           .withColumn('TAX', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('TAX_COST')).otherwise(0))\
                                           .withColumn('TotalCost', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Total_Cost')).otherwise(0))\
                                           .withColumn('NS', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NS_PROCS')).otherwise(0))\
                                           .withColumn('CE', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('CE_PROCS')).otherwise(0))\
                                           .withColumn('RMW', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('RMW_PROCS')).otherwise(0))\
                                           .withColumn('O', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('O_PROCS')).otherwise(0))\
                                           .withColumn('TotalProcs', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Tot_Procs')).otherwise(0))\
                                           .withColumn('NetL', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NET_LOSS')).otherwise(0))\
                                           .withColumn('Count', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')), F.lit(1)).otherwise(0))\
                                          .groupBy('DispYr').agg(F.sum('Count').alias('Loan Count'), 
                                                                F.round(F.sum('UPB_Liquidations'),2).alias('UPB for Liquidations'), 
                                                                F.round((F.sum('Default_UPB')*100/F.sum('ORIG_AMT')),2).alias('Default UPB % of Orig. UPB'), 
                                                                F.round((F.sum('Interest')*100/F.sum('Default_UPB')),2).alias('Interest on Delinquent Loans'), 
                                                                F.round((F.sum('Tot_Liq')*100/F.sum('Default_UPB')),2).alias('Total Liquidition Exp.'), 
                                                                F.round((F.sum('FCC')*100/F.sum('Default_UPB')),2).alias('Foreclosure Costs'), 
                                                                F.round((F.sum('PP')*100/F.sum('Default_UPB')),2).alias('Prop.Pres. Costs'), 
                                                                F.round((F.sum('AR')*100/F.sum('Default_UPB')),2).alias('Asset Recovery Costs'), 
                                                                F.round((F.sum('IE')*100/F.sum('Default_UPB')),2).alias('Miscellaneaous Holding Expenses and Credits'), 
                                                                F.round((F.sum('TAX')*100/F.sum('Default_UPB')),2).alias('Associated Taxes'), 
                                                                F.round((F.sum('TotalCost')*100/F.sum('Default_UPB')),2).alias('Total Costs'), 
                                                                F.round((F.sum('NS')*100/F.sum('Default_UPB')),2).alias('Sales Proceeds'), 
                                                                F.round((F.sum('CE')*100/F.sum('Default_UPB')),2).alias('Credit Enhancement Proceeds'), 
                                                                F.round((F.sum('RMW')*100/F.sum('Default_UPB')),2).alias('Repurchase/Make Whole Proceeds'), 
                                                                F.round((F.sum('O')*100/F.sum('Default_UPB')),2).alias('Other Proceeds'), 
                                                                F.round((F.sum('TotalProcs')*100/F.sum('Default_UPB')),2).alias('Total Proceeds'), 
                                                                F.round((F.sum('NetL')*100/F.sum('Default_UPB')),2).alias('Severity'), 
                                                                F.round((F.sum('NetL')),2).alias('Total Net Loss')).orderBy('DispYr').fillna(0.0)

In [134]:
historical_dispnetloss.toPandas()

,DispYr,Loan Count,UPB for Liquidations,Default UPB % of Orig. UPB,Interest on Delinquent Loans,Total Liquidition Exp.,Foreclosure Costs,Prop.Pres. Costs,Asset Recovery Costs,Miscellaneaous Holding Expenses and Credits,Associated Taxes,Total Costs,Sales Proceeds,Credit Enhancement Proceeds,Repurchase/Make Whole Proceeds,Other Proceeds,Total Proceeds,Severity,Total Net Loss
0,2014,1,150781.94,99.20,0.0,1.61,1.06,0.00,0.00,-1.04,1.59,101.61,101.91,0.86,0.00,1.55,104.32,-2.71,-4084.65
1,2015,17,2488384.48,98.94,0.0,3.44,2.35,0.23,0.00,0.03,0.83,103.44,100.01,4.08,0.00,0.14,104.23,-0.78,-19518.42
2,2016,149,21770208.11,98.09,0.0,5.68,3.27,0.34,0.00,0.52,1.55,105.68,92.77,7.64,0.10,0.37,100.89,4.79,1043802.76
3,2017,353,52852670.35,97.09,0.0,6.57,4.04,0.43,0.00,0.38,1.73,106.57,92.97,7.65,0.00,0.87,101.49,5.09,2688692.88
4,2018,612,87747718.91,96.25,0.0,6.92,4.26,0.34,0.01,0.54,1.76,106.92,96.31,6.93,0.21,0.45,103.90,3.01,2643150.28
5,2019,359,47334777.31,95.98,0.0,7.15,4.20,0.30,0.00,0.69,1.95,107.15,95.38,6.22,0.00,0.30,101.90,5.24,2482553.11


***Historical Net Loss Totals 1***

In [135]:
historical_totalnetloss1 = statistics_data.where(F.col('DispYr') != '9999').where(F.col('DispYr') != 'NO_DISP_DT')

In [136]:
historical_totalnetloss1 = historical_totalnetloss1\
                  .fillna(0.0, ['ORIG_AMT',
                                'ORIG_RT',
                                'LAST_UPB',
                                'INT_COS',
                                'Tot_Liq_Ex',
                                'FCC_COST',
                                'PP_COST',
                                'AR_COST',
                                'IE_COST',
                                'TAX_COST',
                                'Total_Cost',
                                'NS_PROCS',
                                'CE_PROCS',
                                'RMW_PROCS',
                                'O_PROCS',
                                'Tot_Procs',
                                'NET_LOSS']).withColumn('Default_UPB', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Interest', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('INT_COST')).otherwise(0))\
                                           .withColumn('UPB_Liquidations', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('LAST_UPB')).otherwise(0))\
                                           .withColumn('Tot_Liq', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Tot_Liq_Ex')).otherwise(0))\
                                           .withColumn('FCC', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('FCC_COST')).otherwise(0))\
                                           .withColumn('PP', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('PP_COST')).otherwise(0))\
                                           .withColumn('AR', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('AR_COST')).otherwise(0))\
                                           .withColumn('IE', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('IE_COST')).otherwise(0))\
                                           .withColumn('TAX', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('TAX_COST')).otherwise(0))\
                                           .withColumn('TotalCost', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Total_Cost')).otherwise(0))\
                                           .withColumn('NS', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NS_PROCS')).otherwise(0))\
                                           .withColumn('CE', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('CE_PROCS')).otherwise(0))\
                                           .withColumn('RMW', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('RMW_PROCS')).otherwise(0))\
                                           .withColumn('O', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('O_PROCS')).otherwise(0))\
                                           .withColumn('TotalProcs', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('Tot_Procs')).otherwise(0))\
                                           .withColumn('NetL', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')) &
                                                                             (F.col('DISP_DT').isNotNull()), F.col('NET_LOSS')).otherwise(0))\
                                           .withColumn('Count', F.when((F.col('LAST_STAT').isin('F', 'S', 'T', 'N')), F.lit(1)).otherwise(0))\
                                           .withColumn('Total', F.lit('Total'))\
                                          .groupBy('Total').agg(F.sum('Count').alias('Loan Count'), 
                                                                F.round(F.sum('UPB_Liquidations'),2).alias('UPB for Liquidations'), 
                                                                F.round((F.sum('Default_UPB')*100/F.sum('ORIG_AMT')),2).alias('Default UPB % of Orig. UPB'), 
                                                                F.round((F.sum('Interest')*100/F.sum('Default_UPB')),2).alias('Interest on Delinquent Loans'), 
                                                                F.round((F.sum('Tot_Liq')*100/F.sum('Default_UPB')),2).alias('Total Liquidition Exp.'), 
                                                                F.round((F.sum('FCC')*100/F.sum('Default_UPB')),2).alias('Foreclosure Costs'), 
                                                                F.round((F.sum('PP')*100/F.sum('Default_UPB')),2).alias('Prop.Pres. Costs'), 
                                                                F.round((F.sum('AR')*100/F.sum('Default_UPB')),2).alias('Asset Recovery Costs'), 
                                                                F.round((F.sum('IE')*100/F.sum('Default_UPB')),2).alias('Miscellaneaous Holding Expenses and Credits'), 
                                                                F.round((F.sum('TAX')*100/F.sum('Default_UPB')),2).alias('Associated Taxes'), 
                                                                F.round((F.sum('TotalCost')*100/F.sum('Default_UPB')),2).alias('Total Costs'), 
                                                                F.round((F.sum('NS')*100/F.sum('Default_UPB')),2).alias('Sales Proceeds'), 
                                                                F.round((F.sum('CE')*100/F.sum('Default_UPB')),2).alias('Credit Enhancement Proceeds'), 
                                                                F.round((F.sum('RMW')*100/F.sum('Default_UPB')),2).alias('Repurchase/Make Whole Proceeds'), 
                                                                F.round((F.sum('O')*100/F.sum('Default_UPB')),2).alias('Other Proceeds'), 
                                                                F.round((F.sum('TotalProcs')*100/F.sum('Default_UPB')),2).alias('Total Proceeds'), 
                                                                F.round((F.sum('NetL')*100/F.sum('Default_UPB')),2).alias('Severity'), 
                                                                F.round((F.sum('NetL')),2).alias('Total Net Loss')).fillna(0.0)

In [137]:
historical_totalnetloss1.toPandas()

,Total,Loan Count,UPB for Liquidations,Default UPB % of Orig. UPB,Interest on Delinquent Loans,Total Liquidition Exp.,Foreclosure Costs,Prop.Pres. Costs,Asset Recovery Costs,Miscellaneaous Holding Expenses and Credits,Associated Taxes,Total Costs,Sales Proceeds,Credit Enhancement Proceeds,Repurchase/Make Whole Proceeds,Other Proceeds,Total Proceeds,Severity,Total Net Loss
0,Total,1491,212344541.1,96.62,0.0,6.71,4.07,0.35,0.0,0.52,1.76,106.71,94.95,6.99,0.1,0.51,102.55,4.16,8834595.96


***Historical Net Loss Union 1***

In [138]:
historical_total1 = historical_dispnetloss.union(historical_totalnetloss1)

In [139]:
historical_total1.toPandas()

,DispYr,Loan Count,UPB for Liquidations,Default UPB % of Orig. UPB,Interest on Delinquent Loans,Total Liquidition Exp.,Foreclosure Costs,Prop.Pres. Costs,Asset Recovery Costs,Miscellaneaous Holding Expenses and Credits,Associated Taxes,Total Costs,Sales Proceeds,Credit Enhancement Proceeds,Repurchase/Make Whole Proceeds,Other Proceeds,Total Proceeds,Severity,Total Net Loss
0,2014,1,1.507819e+05,99.20,0.0,1.61,1.06,0.00,0.00,-1.04,1.59,101.61,101.91,0.86,0.00,1.55,104.32,-2.71,-4084.65
1,2015,17,2.488384e+06,98.94,0.0,3.44,2.35,0.23,0.00,0.03,0.83,103.44,100.01,4.08,0.00,0.14,104.23,-0.78,-19518.42
2,2016,149,2.177021e+07,98.09,0.0,5.68,3.27,0.34,0.00,0.52,1.55,105.68,92.77,7.64,0.10,0.37,100.89,4.79,1043802.76
3,2017,353,5.285267e+07,97.09,0.0,6.57,4.04,0.43,0.00,0.38,1.73,106.57,92.97,7.65,0.00,0.87,101.49,5.09,2688692.88
4,2018,612,8.774772e+07,96.25,0.0,6.92,4.26,0.34,0.01,0.54,1.76,106.92,96.31,6.93,0.21,0.45,103.90,3.01,2643150.28
5,2019,359,4.733478e+07,95.98,0.0,7.15,4.20,0.30,0.00,0.69,1.95,107.15,95.38,6.22,0.00,0.30,101.90,5.24,2482553.11
6,Total,1491,2.123445e+08,96.62,0.0,6.71,4.07,0.35,0.00,0.52,1.76,106.71,94.95,6.99,0.10,0.51,102.55,4.16,8834595.96


***Calculate Spread at Origination (SATO)***

In [140]:
summary_statistics_sato = combined_data.where(F.col('ORIG_DTE').isNotNull()).withColumn('SATO1', F.col('ORIG_RT')*F.col('ORIG_AMT'))\
                                                                            .withColumn('SATO2', F.col('ORIG_AMT'))\
                                                .groupBy('ORIG_DTE').agg((F.round(F.sum('SATO1')/F.sum('SATO2'), 2)).alias('Avg_NoteRt'))\
                                                    .orderBy(F.substring(F.col('ORIG_DTE'), 4,4), F.substring(F.col('ORIG_DTE'), 1,2))

In [141]:
statistics_data = combined_data.where(F.col('ORIG_DTE').isNotNull()).join(summary_statistics_sato, 'ORIG_DTE', 'left')

In [142]:
statistics_data = statistics_data.withColumn('SATO', F.round(F.col('ORIG_RT') - F.col('Avg_NoteRt'),2))

***Create buckets for continuous attributes, Risk Flag, and group number of borrowers***

In [143]:
statistics_data = statistics_data.withColumn('borrower', F.when((F.col('NUM_BO') == '1') &
                                                                (F.col('NUM_BO').isNotNull()), F.lit(1)).otherwise(F.lit(0)))\
                                 .withColumn('dti', F.when((F.col('DTI').isNotNull()), F.lit(1))\
                                         .otherwise(F.when(F.col('DTI') > 45, F.lit(1))\
                                         .otherwise(F.lit(0))))\
                                 .withColumn('occ', F.when((F.col('OCC_STAT') == '1') &
                                                           (F.col('OCC_STAT').isNotNull()), F.lit(1)).otherwise(F.lit(0)))\
                                 .withColumn('purp', F.when((F.col('PURPOSE') == 'C') &
                                                           (F.col('PURPOSE').isNotNull()), F.lit(1)).otherwise(F.lit(0)))\
                                 .withColumn('RskFctrs', F.col('borrower')+F.col('dti')+F.col('occ')+F.col('purp'))\
                                        .drop('borrower', 'dti', 'occ', 'purp')

In [144]:
statistics_data = statistics_data.withColumn('OcltvBkt', F.when(F.col('OCLTV').isNull(), F.lit('MissingOCLTV'))\
                         .otherwise(F.when((F.col('OCLTV') > 0) & 
                                           (F.col('OCLTV') <= 60), F.lit('0-60'))\
                         .otherwise(F.when((F.col('OCLTV') > 60) & 
                                           (F.col('OCLTV') <= 65), F.lit('60-65'))\
                         .otherwise(F.when((F.col('OCLTV') > 65) & 
                                           (F.col('OCLTV') <= 70), F.lit('65-70'))\
                         .otherwise(F.when((F.col('OCLTV') > 70) & 
                                           (F.col('OCLTV') <= 75), F.lit('70-75'))\
                         .otherwise(F.when((F.col('OCLTV') > 75) & 
                                           (F.col('OCLTV') <= 80), F.lit('75-80'))\
                         .otherwise(F.when((F.col('OCLTV') > 80) & 
                                           (F.col('OCLTV') <= 85), F.lit('80-85'))\
                         .otherwise(F.when((F.col('OCLTV') > 85) & 
                                           (F.col('OCLTV') <= 90), F.lit('85-90'))\
                         .otherwise(F.when((F.col('OCLTV') > 90) & 
                                           (F.col('OCLTV') <= 97), F.lit('90-97'))\
                         .otherwise(F.when((F.col('OCLTV') > 97), F.lit('97+'))))))))))))

In [145]:
statistics_data = statistics_data.withColumn('OltvBkt', F.when(F.col('OLTV').isNull(), F.lit('MissingOLTV'))\
                         .otherwise(F.when((F.col('OLTV') > 0) & 
                                           (F.col('OLTV') <= 60), F.lit('0-60'))\
                         .otherwise(F.when((F.col('OLTV') > 60) & 
                                           (F.col('OLTV') <= 65), F.lit('60-65'))\
                         .otherwise(F.when((F.col('OLTV') > 65) & 
                                           (F.col('OLTV') <= 70), F.lit('65-70'))\
                         .otherwise(F.when((F.col('OLTV') > 70) & 
                                           (F.col('OLTV') <= 75), F.lit('70-75'))\
                         .otherwise(F.when((F.col('OLTV') > 75) & 
                                           (F.col('OLTV') <= 80), F.lit('75-80'))\
                         .otherwise(F.when((F.col('OLTV') > 80) & 
                                           (F.col('OLTV') <= 85), F.lit('80-85'))\
                         .otherwise(F.when((F.col('OLTV') > 85) & 
                                           (F.col('OLTV') <= 90), F.lit('85-90'))\
                         .otherwise(F.when((F.col('OLTV') > 90) & 
                                           (F.col('OLTV') <= 97), F.lit('90-97'))\
                         .otherwise(F.when((F.col('OLTV') > 97), F.lit('97+'))))))))))))

In [146]:
statistics_data = statistics_data.withColumn('FicoBkt', F.when(F.col('CSCORE_MN').isNull(), F.lit('MissingFICO'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 0) & 
                                           (F.col('CSCORE_MN') <= 620), F.lit('0-620'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 620) & 
                                           (F.col('CSCORE_MN') <= 660), F.lit('620-660'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 660) & 
                                           (F.col('CSCORE_MN') <= 700), F.lit('660-700'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 700) & 
                                           (F.col('CSCORE_MN') <= 740), F.lit('700-740'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 740) & 
                                           (F.col('CSCORE_MN') <= 780), F.lit('740-780'))\
                         .otherwise(F.when((F.col('CSCORE_MN') > 780), F.lit('780>')))))))))

In [147]:
statistics_data = statistics_data.withColumn('DtiBkt', F.when(F.col('DTI').isNull(), F.lit('MissingDTI'))\
                         .otherwise(F.when((F.col('DTI') > 0) & 
                                           (F.col('DTI') <= 20), F.lit('0-20'))\
                         .otherwise(F.when((F.col('DTI') > 20) & 
                                           (F.col('DTI') <= 30), F.lit('20-30'))\
                         .otherwise(F.when((F.col('DTI') > 30) & 
                                           (F.col('DTI') <= 40), F.lit('30-40'))\
                         .otherwise(F.when((F.col('DTI') > 40) & 
                                           (F.col('DTI') <= 45), F.lit('40-45'))\
                         .otherwise(F.when((F.col('DTI') > 45), F.lit('45+'))))))))

In [148]:
statistics_data = statistics_data.withColumn('OrigAmtBkt', F.when(F.col('ORIG_AMT').isNull(), F.lit('MissingORGAMT'))\
                         .otherwise(F.when((F.col('ORIG_AMT') > 0) & 
                                           (F.col('ORIG_AMT') <= 85000), F.lit('0-85k'))\
                         .otherwise(F.when((F.col('ORIG_AMT') > 85000) & 
                                           (F.col('ORIG_AMT') <= 110000), F.lit('85k-110k'))\
                         .otherwise(F.when((F.col('ORIG_AMT') > 110000) & 
                                           (F.col('ORIG_AMT') <= 125000), F.lit('110k-125k'))\
                         .otherwise(F.when((F.col('ORIG_AMT') > 125000) & 
                                           (F.col('ORIG_AMT') <= 150000), F.lit('125k-150k'))\
                         .otherwise(F.when((F.col('ORIG_AMT') > 150000) & 
                                           (F.col('ORIG_AMT') <= 175000), F.lit('150k-175k'))\
                         .otherwise(F.when((F.col('ORIG_AMT') > 175000) & 
                                           (F.col('ORIG_AMT') <= 200000), F.lit('175k-200k'))\
                         .otherwise(F.when((F.col('ORIG_AMT') > 200000) & 
                                           (F.col('ORIG_AMT') <= 417000), F.lit('200k-417k'))\
                         .otherwise(F.when((F.col('ORIG_AMT') > 417000), F.lit('417k+')))))))))))

In [149]:
statistics_data = statistics_data.withColumn('NumBoBkt', F.when(F.col('NUM_BO').isNull(), F.lit('MissingNumBo'))\
                         .otherwise(F.when((F.col('NUM_BO') == 1), F.lit('1'))\
                         .otherwise(F.when((F.col('NUM_BO') == 2), F.lit('2')).otherwise(F.lit('3+')))))

In [150]:
statistics_data = statistics_data.withColumn('SATOBkt', F.when(F.col('SATO').isNull(), F.lit('MissingSATO'))\
                         .otherwise(F.when((F.col('SATO') <= (-2)), F.lit('<-2%'))\
                         .otherwise(F.when((F.col('SATO') > (-2)) & 
                                           (F.col('SATO') <= (-1.5)), F.lit('-2%, -1.5%'))\
                         .otherwise(F.when((F.col('SATO') > (-1.5)) & 
                                           (F.col('SATO') <= (-1)), F.lit('-1.5%, -1%'))\
                         .otherwise(F.when((F.col('SATO') > (-1)) & 
                                           (F.col('SATO') <= (-0.5)), F.lit('-1%, -0.5%'))\
                         .otherwise(F.when((F.col('SATO') > (-0.5)) & 
                                           (F.col('SATO') <= 0), F.lit('-0.5%, 0%'))\
                         .otherwise(F.when((F.col('SATO') > 0) & 
                                           (F.col('SATO') <= 0.5), F.lit('0%, 0.5%'))\
                         .otherwise(F.when((F.col('SATO') > 0.5) & 
                                           (F.col('SATO') <= 1), F.lit('0.5%, 1%'))\
                         .otherwise(F.when((F.col('SATO') > 1) & 
                                           (F.col('SATO') <= 1.5), F.lit('1%, 1.5%'))\
                         .otherwise(F.when((F.col('SATO') > 1.5) & 
                                           (F.col('SATO') <= 2), F.lit('1.5%, 2%'))\
                         .otherwise(F.when((F.col('SATO') > 2), F.lit('2%>')))))))))))))

***For the following calculations we need a subset of the data. To speed up the process we are going to rely on this small dataset***

In [151]:
statistics_data_default = statistics_data.where((F.col('DISP_DT').isNotNull()))

In [152]:
statistics_data_default.count()

9362516

***The following section will calculate the default, the severity and loss rates across various dimensions***

***Default by Vintage and Occupancy***

In [153]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [154]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [155]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [156]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [157]:
df = df.where(F.col('OCC_STAT').isNotNull()).groupBy('OCC_STAT').agg(*expr)

In [158]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [159]:
total = df.union(df_total)

In [160]:
total.toPandas()

,OCC_STAT,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,S,0.000000,11.888814,4825.207807,5.727511e+04,7.034634e+04,8.206753e+04,7.959063e+04,6.963359e+04,6993.466891
1,I,0.544574,7.626266,11196.040716,1.194433e+05,1.422361e+05,1.469366e+05,1.434381e+05,1.208658e+05,12608.074604
2,P,0.000000,268.982743,87661.914347,1.070124e+06,1.416911e+06,1.832491e+06,1.549847e+06,1.319145e+06,141124.984517
3,Total,0.544574,288.497824,103683.162870,1.246842e+06,1.629493e+06,2.061495e+06,1.772876e+06,1.509645e+06,160726.526012


***Severity by Vintage and Occupancy***

In [161]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [162]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [163]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('LAST_UPB')).otherwise(F.lit(0)))

In [164]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [165]:
df = df.where(F.col('OCC_STAT').isNotNull()).groupBy('OCC_STAT').agg(*expr)

In [166]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [167]:
total = df.union(df_total)

In [168]:
total.toPandas()

,OCC_STAT,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,S,0.0,25.0,5885.172226,6.825206e+04,8.176822e+04,9.271781e+04,8.670403e+04,7.317299e+04,7228.0
1,I,1.0,10.0,13408.149329,1.400665e+05,1.621366e+05,1.623511e+05,1.532510e+05,1.253040e+05,12904.0
2,P,0.0,709.0,103552.432657,1.235490e+06,1.615111e+06,2.042404e+06,1.664091e+06,1.369851e+06,144570.0
3,Total,1.0,744.0,122845.754212,1.443809e+06,1.859015e+06,2.297473e+06,1.904046e+06,1.568328e+06,164702.0


***Loss Rate by Vintage and Occupancy***

In [169]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [170]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [171]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [172]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [173]:
df = df.where(F.col('OCC_STAT').isNotNull()).groupBy('OCC_STAT').agg(*expr)

In [174]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [175]:
total = df.union(df_total)

In [176]:
total.toPandas()

,OCC_STAT,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,S,0.000000,11.888814,4824.403203,5.726462e+04,7.034376e+04,8.206154e+04,7.958670e+04,6.963158e+04,6993.466891
1,I,0.544574,7.626266,11190.592073,1.194237e+05,1.422234e+05,1.469270e+05,1.434312e+05,1.208658e+05,12608.074604
2,P,0.000000,268.982743,87626.346018,1.069684e+06,1.416523e+06,1.832216e+06,1.549751e+06,1.319133e+06,141124.984517
3,Total,0.544574,288.497824,103641.341294,1.246372e+06,1.629090e+06,2.061204e+06,1.772769e+06,1.509631e+06,160726.526012


***Default Rate by CLTV and occupancy***

In [177]:
df = statistics_data_default

In [178]:
occupancy = [row.OCC_STAT for row in df.select('OCC_STAT').orderBy('OCC_STAT').distinct().collect()]

In [179]:
for occ in occupancy:
    df = df.withColumn(occ, F.when(F.col('OCC_STAT') == occ, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [180]:
expr = [F.sum(x).alias(x) for x in occupancy]

In [181]:
df = df.where(F.col('OcltvBkt').isNotNull()).orderBy('OcltvBkt').groupBy('OcltvBkt').agg(*expr)

In [182]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [183]:
total = df.union(df_total)

In [184]:
total.toPandas()

,OcltvBkt,I,P,S
0,0-60,147664.285945,1.332338e+06,57446.401071
1,60-65,39832.003769,3.402799e+05,13595.752739
2,65-70,87392.872371,5.215018e+05,24376.693395
3,70-75,281225.677964,6.732181e+05,43975.052849
4,75-80,129275.550828,1.998887e+06,130717.238335
5,80-85,11339.182030,2.920364e+05,7422.861284
6,85-90,1.943416,6.588594e+05,93206.673023
7,90-97,0.684391,1.483476e+06,3.090679
8,97+,0.000000,1.169767e+05,0.000000
9,Total,696732.200715,7.417573e+06,370743.763373


***Default Rate by Vintage and Refinance Purpose***

In [185]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [186]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [187]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [188]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [189]:
df = df.where(F.col('PURPOSE').isNotNull()).groupBy('PURPOSE').agg(*expr)

In [190]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [191]:
total = df.union(df_total)

In [192]:
total.toPandas()

,PURPOSE,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,U,0.000000,0.000000,0.000000,1.854387e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
1,C,0.000000,107.130830,22029.979883,2.574140e+05,3.511056e+05,4.537430e+05,4.362981e+05,3.741188e+05,39288.977775
2,R,0.000000,145.955997,22870.532888,2.618182e+05,4.579284e+05,6.475275e+05,3.247144e+05,1.762797e+05,22576.464204
3,P,0.544574,35.410996,58782.650099,7.276079e+05,8.204590e+05,9.602248e+05,1.011863e+06,9.592460e+05,98861.084033
4,Total,0.544574,288.497824,103683.162870,1.246842e+06,1.629493e+06,2.061495e+06,1.772876e+06,1.509645e+06,160726.526012


***Severity by Vintage and Refinance Purpose***

In [193]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [194]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [195]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('LAST_UPB')).otherwise(F.lit(0)))

In [196]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [197]:
df = df.where(F.col('PURPOSE').isNotNull()).groupBy('PURPOSE').agg(*expr)

In [198]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [199]:
total = df.union(df_total)

In [200]:
total.toPandas()

,PURPOSE,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,U,0.0,0.0,0.000000,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1,C,0.0,296.0,26913.258887,3.045210e+05,4.041080e+05,5.054845e+05,4.679812e+05,3.876248e+05,40204.0
2,R,0.0,399.0,28261.156927,3.102612e+05,5.333113e+05,7.340548e+05,3.530814e+05,1.841399e+05,23120.0
3,P,1.0,49.0,67671.338398,8.290247e+05,9.215961e+05,1.057934e+06,1.082984e+06,9.965633e+05,101378.0
4,Total,1.0,744.0,122845.754212,1.443809e+06,1.859015e+06,2.297473e+06,1.904046e+06,1.568328e+06,164702.0


***Loss Rate by Vintage and Refinance Purpose***

In [201]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [202]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [203]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [204]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [205]:
df = df.where(F.col('PURPOSE').isNotNull()).groupBy('PURPOSE').agg(*expr)

In [206]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [207]:
total = df.union(df_total)

In [208]:
total.toPandas()

,PURPOSE,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,U,0.000000,0.000000,0.000000,1.854387e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
1,C,0.000000,107.130830,22018.731150,2.572890e+05,3.510169e+05,4.536801e+05,4.362716e+05,3.741137e+05,39288.977775
2,R,0.000000,145.955997,22862.360834,2.617555e+05,4.578512e+05,6.474824e+05,3.247049e+05,1.762766e+05,22576.464204
3,P,0.544574,35.410996,58760.249310,7.273258e+05,8.202221e+05,9.600415e+05,1.011793e+06,9.592403e+05,98861.084033
4,Total,0.544574,288.497824,103641.341294,1.246372e+06,1.629090e+06,2.061204e+06,1.772769e+06,1.509631e+06,160726.526012


***Default Rate by CLTV and Purpose***

In [209]:
df = statistics_data_default

In [210]:
purpose = [row.PURPOSE for row in df.select('PURPOSE').orderBy('PURPOSE').distinct().collect()]

In [211]:
for purp in purpose:
    df = df.withColumn(purp, F.when(F.col('PURPOSE') == purp, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [212]:
expr = [F.sum(x).alias(x) for x in purpose]

In [213]:
df = df.where(F.col('OcltvBkt').isNotNull()).orderBy('OcltvBkt').groupBy('OcltvBkt').agg(*expr)

In [214]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [215]:
total = df.union(df_total)

In [216]:
total.toPandas()

,OcltvBkt,C,P,R,U
0,0-60,6.449834e+05,3.905129e+05,5.019525e+05,0.000000
1,60-65,1.525905e+05,9.315371e+04,1.479634e+05,0.000000
2,65-70,2.611298e+05,1.645869e+05,2.075547e+05,0.000000
3,70-75,3.427923e+05,3.760810e+05,2.795456e+05,0.000000
4,75-80,5.092671e+05,1.370490e+06,3.791207e+05,1.854387
5,80-85,2.333231e+04,1.509717e+05,1.364945e+05,0.000000
6,85-90,7.376001e+00,5.971345e+05,1.549262e+05,0.000000
7,90-97,2.839696e+00,1.377364e+06,1.061127e+05,0.000000
8,97+,0.000000e+00,1.167859e+05,1.907906e+02,0.000000
9,Total,1.934106e+06,4.637081e+06,1.913861e+06,1.854387


***Default Rate by Vintage and Number of Borrowers***

In [217]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [218]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [219]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [220]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [221]:
df = df.where(F.col('NumBoBkt').isNotNull()).orderBy('NumBoBkt').groupBy('NumBoBkt').agg(*expr)

In [222]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [223]:
total = df.union(df_total)

In [224]:
total.toPandas()

,NumBoBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,0.000000,86.357759,53318.014897,6.308258e+05,8.142874e+05,1.031266e+06,9.456953e+05,8.275875e+05,89793.360567
1,2,0.544574,200.636610,49841.973509,6.103004e+05,8.082072e+05,1.016393e+06,8.110406e+05,6.655452e+05,69164.714336
2,3+,0.000000,1.503454,523.174463,5.715762e+03,6.998396e+03,1.383707e+04,1.613995e+04,1.651182e+04,1768.451109
3,Total,0.544574,288.497824,103683.162870,1.246842e+06,1.629493e+06,2.061495e+06,1.772876e+06,1.509645e+06,160726.526012


***Severity by Vintage and Number of Borrowers***

In [225]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [226]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [227]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('LAST_UPB')).otherwise(F.lit(0)))

In [228]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [229]:
df = df.where(F.col('NumBoBkt').isNotNull()).orderBy('NumBoBkt').groupBy('NumBoBkt').agg(*expr)

In [230]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [231]:
total = df.union(df_total)

In [232]:
total.toPandas()

,NumBoBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,0.0,193.0,62665.672310,7.250748e+05,9.215639e+05,1.142290e+06,1.011105e+06,8.571371e+05,91840.0
1,2,1.0,549.0,59582.081901,7.122781e+05,9.296585e+05,1.140164e+06,8.758669e+05,6.941650e+05,71056.0
2,3+,0.0,2.0,598.000000,6.455936e+03,7.793000e+03,1.501900e+04,1.707400e+04,1.702600e+04,1806.0
3,Total,1.0,744.0,122845.754212,1.443809e+06,1.859015e+06,2.297473e+06,1.904046e+06,1.568328e+06,164702.0


***Loss Rate by Vintage and Number of Borrowers***

In [233]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [234]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [235]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [236]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [237]:
df = df.where(F.col('NumBoBkt').isNotNull()).orderBy('NumBoBkt').groupBy('NumBoBkt').agg(*expr)

In [238]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [239]:
total = df.union(df_total)

In [240]:
total.toPandas()

,NumBoBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,0.000000,86.357759,53287.300051,6.304312e+05,8.139361e+05,1.031004e+06,9.456027e+05,8.275746e+05,89793.360567
1,2,0.544574,200.636610,49830.866780,6.102263e+05,8.081557e+05,1.016363e+06,8.110266e+05,6.655442e+05,69164.714336
2,3+,0.000000,1.503454,523.174463,5.714700e+03,6.998396e+03,1.383707e+04,1.613995e+04,1.651182e+04,1768.451109
3,Total,0.544574,288.497824,103641.341294,1.246372e+06,1.629090e+06,2.061204e+06,1.772769e+06,1.509631e+06,160726.526012


***Default Rate by Vintage and FICO***

In [241]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [242]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [243]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [244]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [245]:
df = df.where(F.col('FicoBkt').isNotNull()).orderBy('FicoBkt').groupBy('FicoBkt').agg(*expr)

In [246]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [247]:
total = df.union(df_total)

In [248]:
total.toPandas()

,FicoBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-620,0.000000,3.342736,136.160224,1.182527e+03,1.435712e+03,1.436981e+03,1.806109e+03,1.760246e+03,179.327749
1,620-660,0.000000,2.117489,5143.711107,7.209522e+04,8.747600e+04,9.704316e+04,1.140842e+05,1.072553e+05,10118.059757
2,660-700,0.000000,19.459329,14247.155173,1.764711e+05,2.139840e+05,2.529975e+05,2.575366e+05,2.288073e+05,24320.557819
3,700-740,0.000000,33.771487,22187.930294,2.655756e+05,3.397362e+05,4.276778e+05,4.000700e+05,3.486213e+05,39201.232214
4,740-780,0.000000,78.201139,31342.354575,3.662588e+05,4.836177e+05,6.126077e+05,5.099860e+05,4.332652e+05,47332.182102
5,780>,0.544574,151.605644,30625.851496,3.652586e+05,5.032433e+05,6.697321e+05,4.893930e+05,3.899353e+05,39575.166372
6,Total,0.544574,288.497824,103683.162870,1.246842e+06,1.629493e+06,2.061495e+06,1.772876e+06,1.509645e+06,160726.526012


***Severity by Vintage and FICO***

In [249]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [250]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [251]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('LAST_UPB')).otherwise(F.lit(0)))

In [252]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [253]:
df = df.where(F.col('FicoBkt').isNotNull()).orderBy('FicoBkt').groupBy('FicoBkt').agg(*expr)

In [254]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [255]:
total = df.union(df_total)

In [256]:
total.toPandas()

,FicoBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-620,0.0,6.0,156.057592,1.337000e+03,1.582853e+03,1.559000e+03,1.908000e+03,1.807000e+03,182.0
1,620-660,0.0,7.0,5959.356883,8.124018e+04,9.694741e+04,1.054697e+05,1.205158e+05,1.101958e+05,10296.0
2,660-700,0.0,41.0,16457.504985,1.990297e+05,2.378483e+05,2.760999e+05,2.727450e+05,2.354139e+05,24769.0
3,700-740,0.0,78.0,25713.492279,3.015814e+05,3.804888e+05,4.694167e+05,4.253747e+05,3.595761e+05,39952.0
4,740-780,0.0,176.0,36822.888771,4.217456e+05,5.493874e+05,6.801104e+05,5.468115e+05,4.494952e+05,48465.0
5,780>,1.0,436.0,37736.453701,4.388750e+05,5.927607e+05,7.648172e+05,5.366915e+05,4.118400e+05,41038.0
6,Total,1.0,744.0,122845.754212,1.443809e+06,1.859015e+06,2.297473e+06,1.904046e+06,1.568328e+06,164702.0


***Loss Rate by Vintage and FICO***

In [257]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [258]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [259]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [260]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [261]:
df = df.where(F.col('FicoBkt').isNotNull()).orderBy('FicoBkt').groupBy('FicoBkt').agg(*expr)

In [262]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [263]:
total = df.union(df_total)

In [264]:
total.toPandas()

,FicoBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-620,0.000000,3.342736,135.251765,1.181553e+03,1.431658e+03,1.436981e+03,1.806109e+03,1.760246e+03,179.327749
1,620-660,0.000000,2.117489,5140.273311,7.202273e+04,8.741172e+04,9.698700e+04,1.140632e+05,1.072502e+05,10118.059757
2,660-700,0.000000,19.459329,14235.138938,1.763234e+05,2.138455e+05,2.529182e+05,2.575089e+05,2.288032e+05,24320.557819
3,700-740,0.000000,33.771487,22177.955413,2.654567e+05,3.396537e+05,4.276054e+05,4.000509e+05,3.486205e+05,39201.232214
4,740-780,0.000000,78.201139,31331.673868,3.661831e+05,4.835511e+05,6.125614e+05,5.099617e+05,4.332633e+05,47332.182102
5,780>,0.544574,151.605644,30621.047998,3.652047e+05,5.031966e+05,6.696951e+05,4.893785e+05,3.899333e+05,39575.166372
6,Total,0.544574,288.497824,103641.341294,1.246372e+06,1.629090e+06,2.061204e+06,1.772769e+06,1.509631e+06,160726.526012


***Default Rate by Vintage and Original Loan Amount***

In [265]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [266]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [267]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [268]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [269]:
df = df.where(F.col('OrigAmtBkt').isNotNull()).orderBy('OrigAmtBkt').groupBy('OrigAmtBkt').agg(*expr)

In [270]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [271]:
total = df.union(df_total)

In [272]:
total.toPandas()

,OrigAmtBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-85k,0.544574,45.861161,11094.173548,1.177927e+05,1.273697e+05,1.380166e+05,1.377663e+05,1.134419e+05,11601.115445
1,110k-125k,0.000000,26.339765,6162.002000,7.138597e+04,8.523996e+04,1.027333e+05,9.561674e+04,7.486463e+04,7479.447573
2,125k-150k,0.000000,38.936404,10276.314021,1.227471e+05,1.494223e+05,1.858177e+05,1.706142e+05,1.356740e+05,13464.360031
3,150k-175k,0.000000,26.263085,9398.613408,1.121453e+05,1.417859e+05,1.790132e+05,1.567510e+05,1.243436e+05,13431.426540
4,175k-200k,0.000000,15.887922,8959.170870,1.127165e+05,1.486342e+05,1.809101e+05,1.610210e+05,1.357817e+05,13506.829849
5,200k-417k,0.000000,93.089405,43668.866318,5.513647e+05,7.826370e+05,1.022158e+06,7.775022e+05,6.780552e+05,73434.947273
6,417k+,0.000000,3.498563,4315.281628,5.088003e+04,7.106951e+04,1.084298e+05,1.380350e+05,1.375305e+05,16858.432159
7,85k-110k,0.000000,38.621517,9808.741077,1.078096e+05,1.233344e+05,1.444166e+05,1.355695e+05,1.099529e+05,10949.967143
8,Total,0.544574,288.497824,103683.162870,1.246842e+06,1.629493e+06,2.061495e+06,1.772876e+06,1.509645e+06,160726.526012


***Severity by Vintage and Original Loan Amount***

In [273]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [274]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [275]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('LAST_UPB')).otherwise(F.lit(0)))

In [276]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [277]:
df = df.where(F.col('OrigAmtBkt').isNotNull()).orderBy('OrigAmtBkt').groupBy('OrigAmtBkt').agg(*expr)

In [278]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [279]:
total = df.union(df_total)

In [280]:
total.toPandas()

,OrigAmtBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-85k,1.0,157.0,14430.272701,1.482568e+05,1.544420e+05,1.599523e+05,1.520316e+05,1.193852e+05,12009.0
1,110k-125k,0.0,72.0,7566.811940,8.513020e+04,9.910957e+04,1.156849e+05,1.032870e+05,7.795714e+04,7676.0
2,125k-150k,0.0,91.0,12438.213748,1.447854e+05,1.728132e+05,2.087622e+05,1.839771e+05,1.411879e+05,13824.0
3,150k-175k,0.0,76.0,11172.241066,1.304162e+05,1.624934e+05,1.999554e+05,1.683518e+05,1.291150e+05,13755.0
4,175k-200k,0.0,42.0,10573.569214,1.301924e+05,1.696709e+05,2.019164e+05,1.727400e+05,1.409160e+05,13850.0
5,200k-417k,0.0,185.0,49759.032763,6.194574e+05,8.774198e+05,1.129714e+06,8.301925e+05,7.027679e+05,75111.0
6,417k+,0.0,4.0,4674.000000,5.478097e+04,7.764638e+04,1.172690e+05,1.458510e+05,1.420110e+05,17187.0
7,85k-110k,0.0,117.0,12231.612780,1.307894e+05,1.454201e+05,1.642184e+05,1.476155e+05,1.149879e+05,11290.0
8,Total,1.0,744.0,122845.754212,1.443809e+06,1.859015e+06,2.297473e+06,1.904046e+06,1.568328e+06,164702.0


***Loss Rate by Vintage and Original Loan Amount***

In [281]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [282]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [283]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [284]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [285]:
df = df.where(F.col('OrigAmtBkt').isNotNull()).orderBy('OrigAmtBkt').groupBy('OrigAmtBkt').agg(*expr)

In [286]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [287]:
total = df.union(df_total)

In [288]:
total.toPandas()

,OrigAmtBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-85k,0.544574,45.861161,11082.400210,1.176660e+05,1.272656e+05,1.379389e+05,1.377343e+05,1.134411e+05,11601.115445
1,110k-125k,0.000000,26.339765,6158.920699,7.134870e+04,8.520193e+04,1.027108e+05,9.560880e+04,7.486378e+04,7479.447573
2,125k-150k,0.000000,38.936404,10272.663321,1.226911e+05,1.493898e+05,1.857894e+05,1.706013e+05,1.356719e+05,13464.360031
3,150k-175k,0.000000,26.263085,9396.922493,1.120967e+05,1.417464e+05,1.789832e+05,1.567419e+05,1.243426e+05,13431.426540
4,175k-200k,0.000000,15.887922,8953.863596,1.126891e+05,1.486078e+05,1.808900e+05,1.610180e+05,1.357817e+05,13506.829849
5,200k-417k,0.000000,93.089405,43659.264905,5.512537e+05,7.825437e+05,1.022098e+06,7.774818e+05,6.780481e+05,73434.947273
6,417k+,0.000000,3.498563,4315.281628,5.087715e+04,7.106500e+04,1.084258e+05,1.380340e+05,1.375305e+05,16858.432159
7,85k-110k,0.000000,38.621517,9802.024440,1.077497e+05,1.232700e+05,1.443684e+05,1.355491e+05,1.099508e+05,10949.967143
8,Total,0.544574,288.497824,103641.341294,1.246372e+06,1.629090e+06,2.061204e+06,1.772769e+06,1.509631e+06,160726.526012


***Default Rate by FICO and Original Loan Amount***

In [289]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [290]:
orig_loan_amounts = [row.OrigAmtBkt for row in df.select('OrigAmtBkt').orderBy('OrigAmtBkt').distinct().collect()]

In [291]:
for ola in orig_loan_amounts:
    df = df.withColumn(ola, F.when(F.col('OrigAmtBkt') == ola, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [292]:
expr = [F.sum(x).alias(x) for x in orig_loan_amounts]

In [293]:
df = df.where(F.col('FicoBkt').isNotNull()).orderBy('FicoBkt').groupBy('FicoBkt').agg(*expr)

In [294]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [295]:
total = df.union(df_total)

In [296]:
total.toPandas()

,FicoBkt,0-85k,110k-125k,125k-150k,150k-175k,175k-200k,200k-417k,417k+,85k-110k
0,0-620,1081.664370,528.783156,882.251089,665.129599,703.771394,2.856587e+03,317.772477,904.447029
1,620-660,64446.517681,32673.932210,54162.723796,46627.930059,43486.783074,1.806391e+05,18851.263574,52329.497972
2,660-700,113224.869419,67860.808673,116735.136218,106761.686089,104837.792745,4.968921e+05,61240.825655,100830.293140
3,700-740,133459.956166,95441.069539,170061.491755,161758.319693,165489.313019,8.600711e+05,124544.499215,132278.070477
4,740-780,158714.394857,119866.040886,217680.480708,210440.368438,220801.062639,1.217512e+06,173974.498492,165499.417708
5,780>,186201.575849,127137.788216,228532.780977,210641.840594,226226.724073,1.170943e+06,148193.187081,190038.581469
6,Total,657128.978341,443508.422680,788054.864543,736895.274472,761545.446944,3.928914e+06,527122.046494,641880.307794


***Default Rate by Vintage and OCLTV***

In [297]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [298]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [299]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [300]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [301]:
df = df.where(F.col('OcltvBkt').isNotNull()).orderBy('OcltvBkt').groupBy('OcltvBkt').agg(*expr)

In [302]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [303]:
total = df.union(df_total)

In [304]:
total.toPandas()

,OcltvBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-60,0.000000,138.671606,16100.895208,1.793149e+05,2.725022e+05,4.196656e+05,3.284734e+05,2.883823e+05,32870.606667
1,60-65,0.000000,24.537763,4360.300100,5.108149e+04,7.757064e+04,1.104777e+05,8.137685e+04,6.200156e+04,6814.550692
2,65-70,0.000000,24.906744,7254.737959,8.552475e+04,1.235662e+05,1.657752e+05,1.315276e+05,1.076748e+05,11923.213294
3,70-75,0.000000,26.358177,13384.877822,1.513946e+05,2.030268e+05,2.515743e+05,2.021409e+05,1.614450e+05,15426.124769
4,75-80,0.544574,51.499441,30624.524965,3.620229e+05,4.413844e+05,5.295207e+05,4.822561e+05,3.787036e+05,34315.754787
5,80-85,0.000000,2.000701,4156.312297,5.797837e+04,6.029662e+04,7.576840e+04,5.997166e+04,4.717864e+04,5446.479978
6,85-90,0.000000,9.568303,9385.006835,1.262887e+05,1.603305e+05,1.708520e+05,1.468912e+05,1.245164e+05,13794.695351
7,90-97,0.000000,10.955089,18020.529800,2.271502e+05,2.754196e+05,3.153696e+05,3.100767e+05,3.002672e+05,37164.854016
8,97+,0.000000,0.000000,395.977884,6.085963e+03,1.539606e+04,2.249183e+04,3.016146e+04,3.947514e+04,2970.246459
9,Total,0.544574,288.497824,103683.162870,1.246842e+06,1.629493e+06,2.061495e+06,1.772876e+06,1.509645e+06,160726.526012


***Severity by Vintage and OCLTV***

In [305]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [306]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [307]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('LAST_UPB')).otherwise(F.lit(0)))

In [308]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [309]:
df = df.where(F.col('OcltvBkt').isNotNull()).orderBy('OcltvBkt').groupBy('OcltvBkt').agg(*expr)

In [310]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [311]:
total = df.union(df_total)

In [312]:
total.toPandas()

,OcltvBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-60,0.0,432.0,21327.065923,2.289715e+05,3.331354e+05,4.896639e+05,3.639478e+05,3.042600e+05,34097.0
1,60-65,0.0,61.0,5433.992622,6.175897e+04,9.112879e+04,1.252660e+05,8.848700e+04,6.482100e+04,7017.0
2,65-70,0.0,59.0,8811.835352,1.008289e+05,1.424191e+05,1.855255e+05,1.416247e+05,1.120600e+05,12242.0
3,70-75,0.0,53.0,15962.702057,1.763658e+05,2.317020e+05,2.793209e+05,2.168223e+05,1.676169e+05,15751.0
4,75-80,1.0,101.0,35757.959883,4.159207e+05,4.999872e+05,5.859171e+05,5.167702e+05,3.935480e+05,35097.0
5,80-85,0.0,7.0,4716.235265,6.475222e+04,6.681101e+04,8.249131e+04,6.351336e+04,4.860798e+04,5543.0
6,85-90,0.0,17.0,10494.786073,1.395679e+05,1.766301e+05,1.858790e+05,1.557360e+05,1.286021e+05,14093.0
7,90-97,0.0,14.0,19907.411999,2.489823e+05,3.005872e+05,3.395178e+05,3.257686e+05,3.086039e+05,37852.0
8,97+,0.0,0.0,433.765038,6.660460e+03,1.661468e+04,2.389131e+04,3.137630e+04,4.020823e+04,3010.0
9,Total,1.0,744.0,122845.754212,1.443809e+06,1.859015e+06,2.297473e+06,1.904046e+06,1.568328e+06,164702.0


***Loss Rate by Vintage and OCLTV***

In [313]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [314]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [315]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [316]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [317]:
df = df.where(F.col('OcltvBkt').isNotNull()).orderBy('OcltvBkt').groupBy('OcltvBkt').agg(*expr)

In [318]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [319]:
total = df.union(df_total)

In [320]:
total.toPandas()

,OcltvBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-60,0.000000,138.671606,16098.162603,1.792768e+05,2.724796e+05,4.196434e+05,3.284624e+05,2.883813e+05,32870.606667
1,60-65,0.000000,24.537763,4359.361497,5.107258e+04,7.755736e+04,1.104739e+05,8.137685e+04,6.200156e+04,6814.550692
2,65-70,0.000000,24.906744,7252.640141,8.549160e+04,1.235504e+05,1.657660e+05,1.315214e+05,1.076738e+05,11923.213294
3,70-75,0.000000,26.358177,13380.812411,1.513532e+05,2.029981e+05,2.515498e+05,2.021294e+05,1.614439e+05,15426.124769
4,75-80,0.544574,51.499441,30613.197942,3.619115e+05,4.412779e+05,5.294550e+05,4.822355e+05,3.787006e+05,34315.754787
5,80-85,0.000000,2.000701,4153.654292,5.795185e+04,6.028411e+04,7.575600e+04,5.996709e+04,4.717763e+04,5446.479978
6,85-90,0.000000,9.568303,9382.891614,1.262382e+05,1.602849e+05,1.708218e+05,1.468853e+05,1.245134e+05,13794.695351
7,90-97,0.000000,10.955089,18005.828532,2.269968e+05,2.752776e+05,3.152666e+05,3.100366e+05,3.002641e+05,37164.854016
8,97+,0.000000,0.000000,394.792261,6.079582e+03,1.538023e+04,2.247158e+04,3.015484e+04,3.947437e+04,2970.246459
9,Total,0.544574,288.497824,103641.341294,1.246372e+06,1.629090e+06,2.061204e+06,1.772769e+06,1.509631e+06,160726.526012


***Default Rate by Vintage and DTI***

In [321]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [322]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [323]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [324]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [325]:
df = df.where(F.col('DtiBkt').isNotNull()).orderBy('DtiBkt').groupBy('DtiBkt').agg(*expr)

In [326]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [327]:
total = df.union(df_total)

In [328]:
total.toPandas()

,DtiBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-20,0.000000,57.214646,8892.229618,1.053474e+05,1.487719e+05,2.061770e+05,1.373571e+05,9.282414e+04,9567.560268
1,20-30,0.000000,101.667079,25382.797410,3.114693e+05,4.279883e+05,5.573276e+05,4.091878e+05,2.894267e+05,30548.386397
2,30-40,0.000000,83.622923,39108.875672,4.769381e+05,6.195006e+05,7.634925e+05,6.365236e+05,4.802495e+05,53221.503784
3,40-45,0.544574,43.000348,25826.366389,3.064045e+05,3.780676e+05,4.501460e+05,4.102190e+05,3.152157e+05,36356.164119
4,45+,0.000000,2.992827,4455.322108,4.636047e+04,5.479808e+04,8.407622e+04,1.792873e+05,3.314479e+05,30982.719306
5,MissingDTI,0.000000,0.000000,17.571673,3.221622e+02,3.666134e+02,2.759021e+02,3.010563e+02,4.806421e+02,50.192137
6,Total,0.544574,288.497824,103683.162870,1.246842e+06,1.629493e+06,2.061495e+06,1.772876e+06,1.509645e+06,160726.526012


***Severity by Vintage and DTI***

In [329]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [330]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [331]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('LAST_UPB')).otherwise(F.lit(0)))

In [332]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [333]:
df = df.where(F.col('DtiBkt').isNotNull()).orderBy('DtiBkt').groupBy('DtiBkt').agg(*expr)

In [334]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [335]:
total = df.union(df_total)

In [336]:
total.toPandas()

,DtiBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-20,0.0,176.0,11646.670968,1.329613e+05,1.813436e+05,2.409283e+05,1.534460e+05,9.898000e+04,9984.0
1,20-30,0.0,271.0,30675.589675,3.666447e+05,4.944951e+05,6.267017e+05,4.431701e+05,3.023172e+05,31401.0
2,30-40,0.0,201.0,45615.487140,5.452858e+05,6.991169e+05,8.438370e+05,6.807849e+05,4.981341e+05,54464.0
3,40-45,1.0,92.0,29665.095787,3.452151e+05,4.215629e+05,4.925521e+05,4.360041e+05,3.258839e+05,37147.0
4,45+,0.0,4.0,5220.910641,5.332793e+04,6.208096e+04,9.315183e+04,1.903253e+05,3.425178e+05,31655.0
5,MissingDTI,0.0,0.0,22.000000,3.740000e+02,4.160000e+02,3.020000e+02,3.160000e+02,4.950000e+02,51.0
6,Total,1.0,744.0,122845.754212,1.443809e+06,1.859015e+06,2.297473e+06,1.904046e+06,1.568328e+06,164702.0


***Loss Rate by Vintage and DTI***

In [337]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [338]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [339]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [340]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [341]:
df = df.where(F.col('DtiBkt').isNotNull()).orderBy('DtiBkt').groupBy('DtiBkt').agg(*expr)

In [342]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [343]:
total = df.union(df_total)

In [344]:
total.toPandas()

,DtiBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0-20,0.000000,57.214646,8888.152446,1.053243e+05,1.487504e+05,2.061658e+05,1.373550e+05,9.282414e+04,9567.560268
1,20-30,0.000000,101.667079,25375.006271,3.113847e+05,4.279115e+05,5.572764e+05,4.091681e+05,2.894259e+05,30548.386397
2,30-40,0.000000,83.622923,39095.176669,4.767412e+05,6.193339e+05,7.633792e+05,6.364799e+05,4.802456e+05,53221.503784
3,40-45,0.544574,43.000348,25811.135544,3.062471e+05,3.779393e+05,4.500434e+05,4.101914e+05,3.152126e+05,36356.164119
4,45+,0.000000,2.992827,4454.298692,4.635268e+04,5.478842e+04,8.406342e+04,1.792738e+05,3.314418e+05,30982.719306
5,MissingDTI,0.000000,0.000000,17.571673,3.221622e+02,3.666134e+02,2.759021e+02,3.010563e+02,4.806421e+02,50.192137
6,Total,0.544574,288.497824,103641.341294,1.246372e+06,1.629090e+06,2.061204e+06,1.772769e+06,1.509631e+06,160726.526012


***Default Rate by Vintage and SATO***

In [345]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [346]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [347]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [348]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [349]:
df = df.where(F.col('SATOBkt').isNotNull()).orderBy('SATOBkt').groupBy('SATOBkt').agg(*expr)

In [350]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [351]:
total = df.union(df_total)

In [352]:
total.toPandas()

,SATOBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"-0.5%, 0%",0.544574,144.590956,28790.521718,3.733945e+05,5.190349e+05,7.109987e+05,6.456805e+05,6.592116e+05,69244.486791
1,"-1%, -0.5%",0.000000,5.958778,10977.076679,1.172392e+05,1.629865e+05,2.411476e+05,1.855894e+05,1.116126e+05,12179.436628
2,"-1.5%, -1%",0.000000,0.000000,4326.278078,4.698085e+04,3.216508e+04,3.232860e+04,3.621046e+04,1.049055e+04,1666.494391
3,"-2%, -1.5%",0.000000,0.000000,154.477818,8.411223e+02,2.201078e+02,1.737386e+02,6.779870e+02,4.070169e+02,66.559560
4,"0%, 0.5%",0.000000,125.062115,47585.717363,5.790712e+05,7.073666e+05,7.891131e+05,6.656349e+05,5.033757e+05,53113.795972
5,"0.5%, 1%",0.000000,8.602453,10358.913738,1.220344e+05,1.879826e+05,2.392404e+05,2.001045e+05,1.846401e+05,18018.292689
6,"1%, 1.5%",0.000000,3.389626,1435.369792,7.175959e+03,1.949042e+04,4.634320e+04,3.673351e+04,3.598842e+04,5839.175469
7,"1.5%, 2%",0.000000,0.893894,46.774392,1.013019e+02,2.449643e+02,2.099911e+03,2.158810e+03,3.890874e+03,593.455167
8,2%>,0.000000,0.000000,0.000000,0.000000e+00,1.876695e+00,4.997149e+01,8.422968e+01,1.847751e+01,1.986574
9,<-2%,0.000000,0.000000,8.033290,3.270642e+00,0.000000e+00,0.000000e+00,1.539289e+00,9.155349e+00,2.842771


***Severity by Vintage and SATO***

In [353]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [354]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [355]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('LAST_UPB')).otherwise(F.lit(0)))

In [356]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [357]:
df = df.where(F.col('SATOBkt').isNotNull()).orderBy('SATOBkt').groupBy('SATOBkt').agg(*expr)

In [358]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [359]:
total = df.union(df_total)

In [360]:
total.toPandas()

,SATOBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"-0.5%, 0%",1.0,415.0,33556.628970,4.271889e+05,5.926879e+05,7.903024e+05,6.915239e+05,6.860850e+05,71010.0
1,"-1%, -0.5%",0.0,14.0,16384.657413,1.681718e+05,2.227386e+05,3.065659e+05,2.177216e+05,1.217450e+05,12768.0
2,"-1.5%, -1%",0.0,0.0,6989.018637,7.246426e+04,4.574742e+04,4.218000e+04,4.369300e+04,1.169700e+04,1778.0
3,"-2%, -1.5%",0.0,0.0,303.000000,1.536000e+03,3.100000e+02,2.290000e+02,8.460000e+02,4.480000e+02,70.0
4,"0%, 0.5%",0.0,298.0,52674.653828,6.347167e+05,7.740411e+05,8.523564e+05,7.013088e+05,5.185340e+05,54199.0
5,"0.5%, 1%",0.0,11.0,11311.707181,1.318803e+05,2.023801e+05,2.547054e+05,2.086248e+05,1.890750e+05,18335.0
6,"1%, 1.5%",0.0,5.0,1562.088183,7.734865e+03,2.084528e+04,4.888108e+04,3.801222e+04,3.675099e+04,5936.0
7,"1.5%, 2%",0.0,1.0,51.000000,1.100000e+02,2.630000e+02,2.200686e+03,2.225981e+03,3.964000e+03,601.0
8,2%>,0.0,0.0,0.000000,0.000000e+00,2.000000e+00,5.200000e+01,8.800000e+01,1.900000e+01,2.0
9,<-2%,0.0,0.0,13.000000,6.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+01,3.0


***Loss Rate by Vintage and SATO***

In [361]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [362]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [363]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [364]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [365]:
df = df.where(F.col('SATOBkt').isNotNull()).orderBy('SATOBkt').groupBy('SATOBkt').agg(*expr)

In [366]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [367]:
total = df.union(df_total)

In [368]:
total.toPandas()

,SATOBkt,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"-0.5%, 0%",0.544574,144.590956,28786.493167,3.733201e+05,5.189693e+05,7.109506e+05,6.456626e+05,6.592096e+05,69244.486791
1,"-1%, -0.5%",0.000000,5.958778,10975.006779,1.172085e+05,1.629718e+05,2.411439e+05,1.855841e+05,1.116126e+05,12179.436628
2,"-1.5%, -1%",0.000000,0.000000,4325.639013,4.697547e+04,3.216179e+04,3.232860e+04,3.621046e+04,1.049055e+04,1666.494391
3,"-2%, -1.5%",0.000000,0.000000,154.477818,8.411223e+02,2.201078e+02,1.737386e+02,6.779870e+02,4.070169e+02,66.559560
4,"0%, 0.5%",0.000000,125.062115,47564.300534,5.788269e+05,7.071677e+05,7.889962e+05,6.655872e+05,5.033707e+05,53113.795972
5,"0.5%, 1%",0.000000,8.602453,10348.021610,1.219264e+05,1.878777e+05,2.391487e+05,2.000805e+05,1.846342e+05,18018.292689
6,"1%, 1.5%",0.000000,3.389626,1432.594690,7.169032e+03,1.947507e+04,4.631376e+04,3.672283e+04,3.598743e+04,5839.175469
7,"1.5%, 2%",0.000000,0.893894,46.774392,1.013019e+02,2.449643e+02,2.098621e+03,2.157796e+03,3.890874e+03,593.455167
8,2%>,0.000000,0.000000,0.000000,0.000000e+00,1.876695e+00,4.997149e+01,8.422968e+01,1.847751e+01,1.986574
9,<-2%,0.000000,0.000000,8.033290,3.270642e+00,0.000000e+00,0.000000e+00,1.539289e+00,9.155349e+00,2.842771


***Calculate SATO by LTV and FICO***

In [369]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [370]:
fico = [row.FicoBkt for row in df.select('FicoBkt').orderBy('FicoBkt').distinct().collect()]

In [371]:
for fic in fico:
    df = df.withColumn(fic, F.when(F.col('FicoBkt') == fic, (F.col('SATO')*F.col('ORIG_AMT'))/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [372]:
expr = [F.sum(x).alias(x) for x in fico]

In [373]:
df = df.where(F.col('OcltvBkt').isNotNull()).orderBy('OcltvBkt').groupBy('OcltvBkt').agg(*expr)

In [374]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [375]:
total = df.union(df_total)

In [376]:
total.toPandas()

,OcltvBkt,0-620,620-660,660-700,700-740,740-780,780>
0,0-60,236.29,8699.83,-16347.73,-49343.82,-110319.47,-227608.009999
1,60-65,171.31,7137.59,2730.61,-6065.73,-21247.16,-36130.680000
2,65-70,416.93,19936.07,14551.45,14.14,-19147.36,-37151.610000
3,70-75,496.01,29316.26,41205.71,26522.58,6384.59,-15265.620000
4,75-80,1148.80,67277.10,103799.31,40977.95,-36408.65,-64257.360000
5,80-85,96.30,5512.64,10649.48,6702.28,-2252.98,-3974.700000
6,85-90,223.66,13492.39,24957.50,18246.26,-2859.00,-6662.740000
7,90-97,491.17,33995.87,73173.75,70220.88,30428.96,7929.780000
8,97+,31.92,6726.10,16011.51,18024.92,11933.02,4404.550000
9,Total,3312.39,192093.85,270731.59,125299.46,-143488.05,-378716.389999


***Calculate Note Rate by LTV and FICO***

In [377]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [378]:
fico = [row.FicoBkt for row in df.select('FicoBkt').orderBy('FicoBkt').distinct().collect()]

In [379]:
for fic in fico:
    df = df.withColumn(fic, F.when(F.col('FicoBkt') == fic, (F.col('ORIG_RT')*F.col('ORIG_AMT'))/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [380]:
expr = [F.sum(x).alias(x) for x in fico]

In [381]:
df = df.where(F.col('OcltvBkt').isNotNull()).orderBy('OcltvBkt').groupBy('OcltvBkt').agg(*expr)

In [382]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [383]:
total = df.union(df_total)

In [384]:
total.toPandas()

,OcltvBkt,0-620,620-660,660-700,700-740,740-780,780>
0,0-60,8858.871,476146.978,821088.107,1130804.916,1.729433e+06,2.779222e+06
1,60-65,2527.555,131502.829,243650.958,328324.454,4.809961e+05,5.841691e+05
2,65-70,4612.778,267174.258,460660.335,574691.943,7.511412e+05,8.264597e+05
3,70-75,5020.213,297051.058,641266.713,916054.750,1.374223e+06,1.414812e+06
4,75-80,10295.129,645653.529,1499246.827,2258548.356,2.956362e+06,3.049972e+06
5,80-85,1087.718,69202.255,200737.745,352526.516,4.502558e+05,3.363459e+05
6,85-90,2113.965,148557.697,459819.122,832013.441,1.102982e+06,8.771042e+05
7,90-97,4403.011,331713.670,1079134.163,1896236.632,2.212298e+06,1.336096e+06
8,97+,393.615,58036.858,148408.228,184829.831,1.355621e+05,5.422076e+04
9,Total,39312.855,2425039.132,5554012.198,8474030.839,1.119325e+07,1.125840e+07


***Default Rate by Vintage and Risk Factors***

In [385]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [386]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [387]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [388]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [389]:
df = df.where(F.col('RskFctrs').isNotNull()).orderBy('RskFctrs').groupBy('RskFctrs').agg(*expr)

In [390]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [391]:
total = df.union(df_total)

In [392]:
total.toPandas()

,RskFctrs,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,0.000000,0.000000,2.454047,4.230188e+01,4.070044e+01,2.361462e+01,2.519089e+01,5.480677e+01,1.951724
1,1,0.544574,120.821848,39511.244257,4.875739e+05,6.351961e+05,7.979800e+05,6.212125e+05,5.178964e+05,54425.602872
2,2,0.000000,141.863361,53006.051976,6.304916e+05,8.234454e+05,1.042227e+06,9.215588e+05,7.821063e+05,83563.844903
3,3,0.000000,25.812614,11163.412589,1.287342e+05,1.708109e+05,2.212646e+05,2.300794e+05,2.095871e+05,22735.126512
4,Total,0.544574,288.497824,103683.162870,1.246842e+06,1.629493e+06,2.061495e+06,1.772876e+06,1.509645e+06,160726.526012


***Severity by Vintage and Risk Factors***

In [393]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [394]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [395]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('LAST_UPB')).otherwise(F.lit(0)))

In [396]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [397]:
df = df.where(F.col('RskFctrs').isNotNull()).orderBy('RskFctrs').groupBy('RskFctrs').agg(*expr)

In [398]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [399]:
total = df.union(df_total)

In [400]:
total.toPandas()

,RskFctrs,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,0.0,0.0,4.000000,5.100000e+01,4.500000e+01,2.600000e+01,2.700000e+01,5.700000e+01,2.0
1,1,1.0,320.0,46619.523325,5.637824e+05,7.271315e+05,8.938405e+05,6.706488e+05,5.405860e+05,55944.0
2,2,0.0,359.0,62883.530577,7.306781e+05,9.383769e+05,1.159714e+06,9.879435e+05,8.110462e+05,85517.0
3,3,0.0,65.0,13338.700310,1.492974e+05,1.934620e+05,2.438923e+05,2.454271e+05,2.166388e+05,23239.0
4,Total,1.0,744.0,122845.754212,1.443809e+06,1.859015e+06,2.297473e+06,1.904046e+06,1.568328e+06,164702.0


***Loss Rate by Vintage and Risk Factors***

In [401]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [402]:
vintage_years = [row.VinYr for row in df.select('VinYr').orderBy('VinYr').distinct().collect() if row.VinYr != '9999']

In [403]:
for vinyr in vintage_years:
    df = df.withColumn(vinyr, F.when(F.col('VinYr') == vinyr, F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [404]:
expr = [F.sum(x).alias(x) for x in vintage_years]

In [405]:
df = df.where(F.col('RskFctrs').isNotNull()).orderBy('RskFctrs').groupBy('RskFctrs').agg(*expr)

In [406]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [407]:
total = df.union(df_total)

In [408]:
total.toPandas()

,RskFctrs,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,0.000000,0.000000,2.454047,4.230188e+01,4.070044e+01,2.361462e+01,2.519089e+01,5.480677e+01,1.951724
1,1,0.544574,120.821848,39503.408175,4.875147e+05,6.351550e+05,7.979571e+05,6.212034e+05,5.178953e+05,54425.602872
2,2,0.000000,141.863361,52980.044569,6.301899e+05,8.231619e+05,1.042015e+06,9.214830e+05,7.820985e+05,83563.844903
3,3,0.000000,25.812614,11155.434503,1.286252e+05,1.707326e+05,2.212083e+05,2.300577e+05,2.095820e+05,22735.126512
4,Total,0.544574,288.497824,103641.341294,1.246372e+06,1.629090e+06,2.061204e+06,1.772769e+06,1.509631e+06,160726.526012


***Default Rate by LTV and FICO***

In [409]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [410]:
fico = [row.FicoBkt for row in df.select('FicoBkt').orderBy('FicoBkt').distinct().collect()]

In [411]:
for fic in fico:
    df = df.withColumn(fic, F.when(F.col('FicoBkt') == fic, F.col('LAST_UPB')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [412]:
expr = [F.sum(x).alias(x) for x in fico]

In [413]:
df = df.where(F.col('OcltvBkt').isNotNull()).orderBy('OcltvBkt').groupBy('OcltvBkt').agg(*expr)

In [414]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [415]:
total = df.union(df_total)

In [416]:
total.toPandas()

,OcltvBkt,0-620,620-660,660-700,700-740,740-780,780>
0,0-60,1874.090493,101025.436561,1.793207e+05,2.506461e+05,3.861938e+05,6.183887e+05
1,60-65,515.390377,27277.705269,5.265384e+04,7.272945e+04,1.086562e+05,1.318750e+05
2,65-70,930.068550,54761.184112,9.851298e+04,1.261815e+05,1.679663e+05,1.849193e+05
3,70-75,994.806715,59175.290119,1.328713e+05,1.963826e+05,3.003632e+05,3.086316e+05
4,75-80,2027.812032,128474.537026,3.098062e+05,4.908242e+05,6.576578e+05,6.700893e+05
5,80-85,221.057809,14168.911318,4.250418e+04,7.745589e+04,1.011677e+05,7.528076e+04
6,85-90,422.237780,30147.885757,9.744642e+04,1.825896e+05,2.471259e+05,1.943359e+05
7,90-97,876.435134,66802.792458,2.257682e+05,4.090836e+05,4.877148e+05,2.932338e+05
8,97+,78.507137,11384.020126,2.949968e+04,3.721087e+04,2.764248e+04,1.116111e+04
9,Total,7940.406027,493217.762747,1.168384e+06,1.843104e+06,2.484488e+06,2.487915e+06


***Severity Rate by LTV and FICO***

In [417]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [418]:
fico = [row.FicoBkt for row in df.select('FicoBkt').orderBy('FicoBkt').distinct().collect()]

In [419]:
for fic in fico:
    df = df.withColumn(fic, F.when(F.col('FicoBkt') == fic, F.col('NET_LOSS')/F.col('LAST_UPB')).otherwise(F.lit(0)))

In [420]:
expr = [F.sum(x).alias(x) for x in fico]

In [421]:
df = df.where(F.col('OcltvBkt').isNotNull()).orderBy('OcltvBkt').groupBy('OcltvBkt').agg(*expr)

In [422]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [423]:
total = df.union(df_total)

In [424]:
total.toPandas()

,OcltvBkt,0-620,620-660,660-700,700-740,740-780,780>
0,0-60,2033.969829,110524.846254,1.995848e+05,2.834590e+05,4.454432e+05,7.347889e+05
1,60-65,560.000000,29674.361225,5.790528e+04,8.092488e+04,1.224529e+05,1.524564e+05
2,65-70,993.000000,58752.613649,1.067446e+05,1.385854e+05,1.871661e+05,2.113293e+05
3,70-75,1077.000000,63969.048510,1.441599e+05,2.146324e+05,3.315929e+05,3.481634e+05
4,75-80,2175.011825,138093.018570,3.347664e+05,5.333588e+05,7.219218e+05,7.527852e+05
5,80-85,235.928229,15241.107386,4.565774e+04,8.331177e+04,1.094737e+05,8.252188e+04
6,85-90,451.000000,32181.617895,1.041414e+05,1.954619e+05,2.660667e+05,2.127172e+05
7,90-97,928.000000,70370.062877,2.386978e+05,4.336031e+05,5.199497e+05,3.176845e+05
8,97+,84.000000,11824.596041,3.074615e+04,3.884393e+04,2.894700e+04,1.174907e+04
9,Total,8537.909883,530631.272407,1.262404e+06,2.002181e+06,2.733014e+06,2.824196e+06


***Loss Rate by LTV and FICO***

In [425]:
df = statistics_data_default.fillna('9999', 'VinYr')

In [426]:
fico = [row.FicoBkt for row in df.select('FicoBkt').orderBy('FicoBkt').distinct().collect()]

In [427]:
for fic in fico:
    df = df.withColumn(fic, F.when(F.col('FicoBkt') == fic, F.col('NET_LOSS')/F.col('ORIG_AMT')).otherwise(F.lit(0)))

In [428]:
expr = [F.sum(x).alias(x) for x in fico]

In [429]:
df = df.where(F.col('OcltvBkt').isNotNull()).orderBy('OcltvBkt').groupBy('OcltvBkt').agg(*expr)

In [430]:
df_total = df.withColumn('Total', F.lit('Total')).groupBy('Total').agg(*expr)

AGGREGATE

In [431]:
total = df.union(df_total)

In [432]:
total.toPandas()

,OcltvBkt,0-620,620-660,660-700,700-740,740-780,780>
0,0-60,1871.141187,101001.134673,1.792879e+05,2.506320e+05,3.861845e+05,6.183742e+05
1,60-65,515.390377,27269.443821,5.264644e+04,7.272468e+04,1.086516e+05,1.318732e+05
2,65-70,930.068550,54742.329184,9.849164e+04,1.261707e+05,1.679583e+05,1.849110e+05
3,70-75,994.806715,59152.298959,1.328336e+05,1.963602e+05,3.003504e+05,3.086164e+05
4,75-80,2025.875741,128414.553676,3.097007e+05,4.907558e+05,6.576075e+05,6.700571e+05
5,80-85,220.006153,14157.309046,4.248924e+04,7.743592e+04,1.011606e+05,7.527578e+04
6,85-90,422.237780,30126.323698,9.740718e+04,1.825589e+05,2.471064e+05,1.943098e+05
7,90-97,876.435134,66754.137212,2.256334e+05,4.089647e+05,4.876156e+05,2.931790e+05
8,97+,78.507137,11377.719579,2.948418e+04,3.719715e+04,2.762788e+04,1.116022e+04
9,Total,7934.468774,492995.249849,1.167974e+06,1.842800e+06,2.484263e+06,2.487757e+06
